# 4 Modeling, feature selection, training, validaton

In [ ]:
# 4 Modeling, feature selection, training, validaton


In [1]:
import pandas as pd

# Path to your rebalanced dataset
path = '/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv'

# Load data
df = pd.read_csv(path)
print(df.shape)
print(df.columns.tolist())


(7140, 79)
['Date', 'Competition', 'Opponent', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min', ' Gls', ' Ast', ' PK', ' PKatt', ' Sh', ' SoT', ' CrdY', ' CrdR', ' Int', 'Match URL', 'Season', ' Touches', ' Tkl', ' Blocks', 'Expected xG', 'Expected npxG', 'Expected xAG', 'SCA', 'GCA', 'Passes Cmp', 'Passes Att', 'Passes Cmp%', 'Passes PrgP', 'Carries Carries', 'Carries PrgC', 'Take-Ons Att', 'Take-Ons Succ', 'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att', 'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Total Cmp', 'Total Att', 'Total Cmp%', 'Total TotDist', 'Total PrgDist', 'Short Cmp', 'Short Att', 'Short Cmp%', 'Medium Cmp', 'Medium Att', 'Medium Cmp%', 'Long Cmp', 'Long Att', 'Long Cmp%', 'Ast', 'xAG', 'xA', 'KP', '3-Jan', 'PPA', 'CrsPA', 'PrgP', 'SCA SCA', 'SCA GCA', ' 1/3', 'Position_Group', 'Rebalanced_Score']


## 2. Add Weekly ID

In [2]:
# After adding the Week column
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df['Week'] = df['Date'].dt.isocalendar().week
else:
    df['Week'] = (df.index // 10) + 1

# Save the updated DataFrame back to the same file
df.to_csv(path, index=False)
print("Updated CSV with Week column saved!")


Updated CSV with Week column saved!


/var/folders/q2/5yylt10156q8bq2drpk08r7w0000gn/T/ipykernel_24954/1887114791.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("="*80)
print(" POSITION-SPECIFIC ML TRAINING")
print("="*80)

# ==========================================
# DATA LOADING AND PREPARATION
# ==========================================
# Path to your rebalanced dataset
path = '/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv'

# Load data
df = pd.read_csv(path)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Add Week column if it doesn't exist
if 'Week' not in df.columns:
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['Week'] = df['Date'].dt.isocalendar().week
    else:
        # Simulated weeks for demonstration
        df['Week'] = (df.index // 10) + 1
    
    # Save back to file
    df.to_csv(path, index=False)
    print("Week column added and saved!")

print(f"Week range: {df['Week'].min()} - {df['Week'].max()}")
print(f"Position groups: {df['Position_Group'].unique()}")

# ==========================================
# POSITION-SPECIFIC METRIC DEFINITIONS
# ==========================================

# Updated position metrics based on multicollinearity-free variables (REVISED FINAL LIST)
position_metrics = {
    'Forward': {
        'core_metrics': ['Gls', 'Ast', 'SoT', 'Expected xG', 'Expected xAG', 'Take-Ons Succ'],
        'secondary_metrics': ['Min'],
        'description': 'Goal scoring and creativity metrics (multicollinearity-free)'
    },
    'Midfield': {
        'core_metrics': ['Passes Cmp%', 'KP', 'Tkl', 'Carries PrgC', 'Passes PrgP', 'Touches'],
        'secondary_metrics': ['Min'],
        'description': 'Passing, creativity, and defensive contribution metrics (multicollinearity-free)'
    },
    'Defense': {
        'core_metrics': ['Int', 'Blocks', 'Clr', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Blocks Sh', 'Blocks Pass'],
        'secondary_metrics': ['Min'],
        'description': 'Defensive actions and positioning metrics (multicollinearity-free)'
    },
    'Goalkeeper': {
        'core_metrics': ['Total Cmp%', 'Err', 'Total PrgDist', 'Short Cmp%', 'Medium Cmp%', 'Total Cmp', 'Short Att'],
        'secondary_metrics': ['Min'],
        'description': 'Distribution accuracy and consistency metrics (multicollinearity-free)'
    }
}

def get_available_metrics(df, position_metrics_dict):
    """Get metrics that actually exist in the dataset"""
    available_metrics = {}
    for position, metrics in position_metrics_dict.items():
        # Check both core and secondary metrics
        available_core = []
        for m in metrics['core_metrics']:
            if m in df.columns:
                available_core.append(m)
            elif f' {m}' in df.columns:  # Handle space prefix
                available_core.append(f' {m}')
        
        available_secondary = []
        for m in metrics['secondary_metrics']:
            if m in df.columns:
                available_secondary.append(m)
            elif f' {m}' in df.columns:  # Handle space prefix
                available_secondary.append(f' {m}')
        
        available_metrics[position] = {
            'core_metrics': available_core,
            'secondary_metrics': available_secondary,
            'all_metrics': available_core + available_secondary,
            'description': metrics['description']
        }
        
        print(f"\n{position}:")
        print(f"  Available core metrics ({len(available_core)}): {available_core}")
        print(f"  Available secondary metrics ({len(available_secondary)}): {available_secondary}")
    
    return available_metrics

available_metrics = get_available_metrics(df, position_metrics)

# ==========================================
# POSITION-SPECIFIC TRAIN/TEST SPLIT
# ==========================================

def create_position_datasets(df, position, metrics_list, test_weeks=2):
    """
    Create position-specific train/test datasets
    """
    print(f"\n{'='*60}")
    print(f"CREATING DATASETS FOR {position.upper()}")
    print(f"{'='*60}")
    
    # Filter by position and remove rows with NaN in Rebalanced_Score
    position_data = df[(df['Position_Group'] == position) & (df['Rebalanced_Score'].notna())].copy()
    
    if len(position_data) == 0:
        print(f"No data found for {position} with valid Rebalanced_Score")
        return None
    
    print(f"Total {position} observations with valid scores: {len(position_data)}")
    print(f"Unique players: {position_data['Player'].nunique()}")
    print(f"Metrics to use: {len(metrics_list)}")
    
    # Check which metrics are available
    available_metrics_for_pos = [m for m in metrics_list if m in position_data.columns]
    missing_metrics = [m for m in metrics_list if m not in position_data.columns]
    
    print(f"Available metrics ({len(available_metrics_for_pos)}): {available_metrics_for_pos}")
    if missing_metrics:
        print(f"Missing metrics ({len(missing_metrics)}): {missing_metrics}")
    
    if len(available_metrics_for_pos) < 3:
        print(f"Insufficient metrics for {position} (need at least 3)")
        return None
    
    # Time-based split (latest weeks for testing)
    latest_week = position_data['Week'].max()
    test_start_week = latest_week - test_weeks + 1
    
    train_data = position_data[position_data['Week'] < test_start_week]
    test_data = position_data[position_data['Week'] >= test_start_week]
    
    print(f"Training weeks: {train_data['Week'].min()} - {train_data['Week'].max()}")
    print(f"Testing weeks: {test_data['Week'].min()} - {test_data['Week'].max()}")
    print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")
    
    if len(train_data) < 10:
        print(f"Insufficient training data for {position} (need at least 10)")
        return None
    
    if len(test_data) < 1:
        print(f"Insufficient test data for {position} (need at least 1)")
        return None
    
    # Prepare features and target
    X_train = train_data[available_metrics_for_pos].fillna(0)
    y_train = train_data['Rebalanced_Score']
    
    X_test = test_data[available_metrics_for_pos].fillna(0)
    y_test = test_data['Rebalanced_Score']
    
    # Store additional info
    train_players = train_data['Player'].tolist()
    test_players = test_data['Player'].tolist()
    
    return {
        'position': position,
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
        'train_players': train_players,
        'test_players': test_players,
        'metrics_used': available_metrics_for_pos,
        'train_data': train_data,
        'test_data': test_data
    }

# ==========================================
# CREATE DATASETS FOR ALL POSITIONS
# ==========================================

position_datasets = {}

for position in ['Forward', 'Midfield', 'Defense', 'Goalkeeper']:
    if position in available_metrics:
        # Use all available metrics for this position
        metrics_to_use = available_metrics[position]['all_metrics']
        
        dataset = create_position_datasets(df, position, metrics_to_use)
        
        if dataset is not None:
            position_datasets[position] = dataset
        else:
            print(f"Skipping {position} due to insufficient data")

print(f"\nSuccessfully created datasets for {len(position_datasets)} positions: {list(position_datasets.keys())}")

# ==========================================
# TRAIN POSITION-SPECIFIC MODELS
# ==========================================

def train_position_model(dataset_info):
    """
    Train Random Forest model for specific position
    """
    position = dataset_info['position']
    X_train = dataset_info['X_train']
    y_train = dataset_info['y_train']
    X_test = dataset_info['X_test']
    y_test = dataset_info['y_test']
    
    print(f"\nTRAINING MODEL FOR {position.upper()}")
    print("-" * 50)
    
    # Train Random Forest model
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    print(f"MODEL PERFORMANCE:")
    print(f"  Training R²: {train_r2:.3f}")
    print(f"  Testing R²:  {test_r2:.3f}")
    print(f"  Training MAE: {train_mae:.3f}")
    print(f"  Testing MAE:  {test_mae:.3f}")
    print(f"  Training RMSE: {train_rmse:.3f}")
    print(f"  Testing RMSE:  {test_rmse:.3f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'metric': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTOP 5 IMPORTANT METRICS:")
    for _, row in feature_importance.head().iterrows():
        print(f"  {row['metric']}: {row['importance']:.3f}")
    
    return {
        'model': model,
        'feature_importance': feature_importance,
        'metrics': {
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_mae': train_mae,
            'test_mae': test_mae,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse
        },
        'predictions': {
            'y_train_pred': y_train_pred,
            'y_test_pred': y_test_pred
        }
    }

# Train models for each position
position_models = {}

for position, dataset in position_datasets.items():
    model_info = train_position_model(dataset)
    position_models[position] = model_info

print(f"\nAll position-specific models trained successfully!")
print(f"Trained models for: {list(position_models.keys())}")

# ==========================================
# MODEL SUMMARY
# ==========================================

print("\n" + "="*80)
print("POSITION-SPECIFIC MODEL SUMMARY")
print("="*80)

print(f"{'Position':<12} {'Test R²':<10} {'Test MAE':<10} {'Top Metric':<20} {'Importance':<10}")
print("-" * 70)

for position, model_info in position_models.items():
    test_r2 = model_info['metrics']['test_r2']
    test_mae = model_info['metrics']['test_mae']
    top_metric = model_info['feature_importance'].iloc[0]['metric']
    top_importance = model_info['feature_importance'].iloc[0]['importance']
    
    print(f"{position:<12} {test_r2:<10.3f} {test_mae:<10.3f} {top_metric:<20} {top_importance:<10.3f}")

print("\nPosition-specific ML training complete!")

 POSITION-SPECIFIC ML TRAINING
Dataset shape: (7140, 80)
Columns: ['Date', 'Competition', 'Opponent', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min', ' Gls', ' Ast', ' PK', ' PKatt', ' Sh', ' SoT', ' CrdY', ' CrdR', ' Int', 'Match URL', 'Season', ' Touches', ' Tkl', ' Blocks', 'Expected xG', 'Expected npxG', 'Expected xAG', 'SCA', 'GCA', 'Passes Cmp', 'Passes Att', 'Passes Cmp%', 'Passes PrgP', 'Carries Carries', 'Carries PrgC', 'Take-Ons Att', 'Take-Ons Succ', 'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att', 'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Total Cmp', 'Total Att', 'Total Cmp%', 'Total TotDist', 'Total PrgDist', 'Short Cmp', 'Short Att', 'Short Cmp%', 'Medium Cmp', 'Medium Att', 'Medium Cmp%', 'Long Cmp', 'Long Att', 'Long Cmp%', 'Ast', 'xAG', 'xA', 'KP', '3-Jan', 'PPA', 'CrsPA', 'PrgP', 'SCA SCA', 'SCA GCA', ' 1/3', 'Position_Grou

In [5]:
#XGBOOST

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
from pathlib import Path

# Load data
path = '/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv'
df = pd.read_csv(path)
print(f"Data loaded successfully: {df.shape}")


Data loaded successfully: (7140, 80)


In [6]:
# ==========================================
# XGBOOST WITH RAW MATCH STATISTICS ONLY
# ==========================================

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

print("XGBOOST TRAINING WITH RAW MATCH STATISTICS")
print("="*60)

print(f"Dataset shape: {df.shape}")
print(f"Available columns: {list(df.columns)}")

# ==========================================
# SELECT RAW MATCH STATISTICS ONLY
# ==========================================

# Define the raw match statistics you want to use
raw_match_stats = [
    'Min',           # Minutes played
    ' Gls',          # Goals (note the space)
    ' Ast',          # Assists
    ' SoT',          # Shots on target
    ' KP',           # Key passes
    ' Tkl',          # Tackles
    ' Int',          # Interceptions
    ' Blocks',       # Blocks
    ' Clr',          # Clearances
    'Passes Cmp%',   # Pass completion %
    'Expected xG',   # Expected goals
    'Expected xAG',  # Expected assists
    'Take-Ons Succ', # Successful take-ons
    'Carries PrgC',  # Progressive carries
    'Passes PrgP',   # Progressive passes
    'Touches'        # Total touches
]

# Check which features actually exist in your dataset
available_features = []
missing_features = []

for feature in raw_match_stats:
    if feature in df.columns:
        available_features.append(feature)
    else:
        missing_features.append(feature)

print(f"\nAvailable features ({len(available_features)}): {available_features}")
if missing_features:
    print(f"Missing features ({len(missing_features)}): {missing_features}")

# ==========================================
# PREPARE DATA FOR TRAINING
# ==========================================

def prepare_raw_data(df, features, test_weeks=4):
    """Prepare data using only raw match statistics"""
    
    # Remove rows with missing target variable
    df_clean = df[df['Rebalanced_Score'].notna()].copy()
    
    # Create week column if not exists
    if 'Week' not in df_clean.columns:
        df_clean['Date'] = pd.to_datetime(df_clean['Date'])
        df_clean['Week'] = df_clean['Date'].dt.isocalendar().week
    
    print(f"Clean dataset: {len(df_clean)} observations")
    print(f"Week range: {df_clean['Week'].min()} - {df_clean['Week'].max()}")
    
    # Time-based split
    latest_week = df_clean['Week'].max()
    test_start_week = latest_week - test_weeks + 1
    
    train_data = df_clean[df_clean['Week'] < test_start_week]
    test_data = df_clean[df_clean['Week'] >= test_start_week]
    
    print(f"Training weeks: {train_data['Week'].min()} - {train_data['Week'].max()}")
    print(f"Test weeks: {test_data['Week'].min()} - {test_data['Week'].max()}")
    print(f"Train: {len(train_data)}, Test: {len(test_data)}")
    
    # Prepare features and target
    X_train = train_data[features].fillna(0)
    y_train = train_data['Rebalanced_Score']
    X_test = test_data[features].fillna(0)
    y_test = test_data['Rebalanced_Score']
    
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
        'train_data': train_data,
        'test_data': test_data,
        'features': features
    }

# ==========================================
# TRAIN MODELS WITH RAW STATISTICS
# ==========================================

# 1. Combined model (all positions)
print("\n" + "="*50)
print("TRAINING COMBINED MODEL")
print("="*50)

combined_raw_data = prepare_raw_data(df, available_features)

def train_raw_xgb(data_dict, name):
    """Train XGBoost with raw features"""
    print(f"\nTraining {name} XGBoost model...")
    
    X_train = data_dict['X_train']
    y_train = data_dict['y_train']
    X_test = data_dict['X_test']
    y_test = data_dict['y_test']
    
    # XGBoost model
    model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        verbosity=0
    )
    
    # Train
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    print(f"Performance:")
    print(f"  Train - R²: {train_r2:.3f}, MAE: {train_mae:.3f}, RMSE: {train_rmse:.3f}")
    print(f"  Test  - R²: {test_r2:.3f}, MAE: {test_mae:.3f}, RMSE: {test_rmse:.3f}")
    
    # Feature importance
    importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 5 Raw Features:")
    for _, row in importance.head().iterrows():
        print(f"  {row['feature']}: {row['importance']:.3f}")
    
    return {
        'model': model,
        'importance': importance,
        'metrics': {
            'train_r2': train_r2, 'test_r2': test_r2,
            'train_mae': train_mae, 'test_mae': test_mae,
            'train_rmse': train_rmse, 'test_rmse': test_rmse
        },
        'predictions': {'train': y_train_pred, 'test': y_test_pred},
        'actuals': {'train': y_train, 'test': y_test},
        'data': data_dict
    }

# Train combined model
raw_models = {}
raw_models['Combined'] = train_raw_xgb(combined_raw_data, 'Combined')

# 2. Position-specific models with raw features
print("\n" + "="*50)
print("TRAINING POSITION-SPECIFIC MODELS")
print("="*50)

for position in ['Forward', 'Midfield', 'Defense', 'Goalkeeper']:
    position_data = df[df['Position_Group'] == position]
    
    if len(position_data) < 50:  # Need sufficient data
        print(f"Skipping {position} - insufficient data ({len(position_data)} samples)")
        continue
    
    try:
        pos_data = prepare_raw_data(position_data, available_features)
        raw_models[position] = train_raw_xgb(pos_data, position)
    except Exception as e:
        print(f"Error training {position} model: {e}")

# ==========================================
# COMPARISON: RAW vs ENGINEERED FEATURES
# ==========================================

print("\n" + "="*70)
print("RAW STATISTICS MODEL SUMMARY")
print("="*70)

print(f"{'Model':<12} {'Test R²':<10} {'Test MAE':<10} {'Test RMSE':<11} {'Top Feature':<20}")
print("-" * 70)

for name, model_info in raw_models.items():
    metrics = model_info['metrics']
    top_feature = model_info['importance'].iloc[0]['feature']
    
    print(f"{name:<12} {metrics['test_r2']:<10.3f} {metrics['test_mae']:<10.3f} "
          f"{metrics['test_rmse']:<11.3f} {top_feature:<20}")

# ==========================================
# SIMPLE PREDICTION FUNCTION
# ==========================================

def predict_with_raw_features(player_name, model_type='Combined'):
    """Predict using raw match statistics only"""
    
    if model_type not in raw_models:
        print(f"Model {model_type} not available")
        available = list(raw_models.keys())
        print(f"Available models: {available}")
        return None
    
    model_info = raw_models[model_type]
    test_data = model_info['data']['test_data']
    
    # Find player
    player_matches = test_data[test_data['Player'].str.contains(player_name, case=False, na=False)]
    
    if len(player_matches) == 0:
        print(f"Player '{player_name}' not found in {model_type} test data")
        return None
    
    # Use most recent match
    latest_match = player_matches.iloc[-1]
    
    # Get raw features
    features = model_info['data']['features']
    X_latest = latest_match[features].fillna(0).values.reshape(1, -1)
    
    # Predict
    prediction = model_info['model'].predict(X_latest)[0]
    actual = latest_match['Rebalanced_Score']
    
    print(f"\n🎯 {player_name} Prediction (Raw Features - {model_type}):")
    print(f"   Date: {latest_match['Date']}")
    print(f"   Actual score: {actual:.3f}")
    print(f"   Predicted: {prediction:.3f}")
    print(f"   Difference: {abs(actual - prediction):.3f}")
    
    return prediction

print(f"\n✅ Raw statistics models ready!")
print(f"📊 Trained {len(raw_models)} models using only original match data")
print(f"🎯 Use: predict_with_raw_features('Player Name', 'Model Type')")

# Show what raw features are being used
print(f"\n📋 Raw features being used ({len(available_features)}):")
for i, feature in enumerate(available_features):
    print(f"  {i+1:2d}. {feature}")

XGBOOST TRAINING WITH RAW MATCH STATISTICS
Dataset shape: (7140, 80)
Available columns: ['Date', 'Competition', 'Opponent', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min', ' Gls', ' Ast', ' PK', ' PKatt', ' Sh', ' SoT', ' CrdY', ' CrdR', ' Int', 'Match URL', 'Season', ' Touches', ' Tkl', ' Blocks', 'Expected xG', 'Expected npxG', 'Expected xAG', 'SCA', 'GCA', 'Passes Cmp', 'Passes Att', 'Passes Cmp%', 'Passes PrgP', 'Carries Carries', 'Carries PrgC', 'Take-Ons Att', 'Take-Ons Succ', 'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd', 'Tackles Att 3rd', 'Challenges Tkl', 'Challenges Att', 'Challenges Tkl%', 'Challenges Lost', 'Blocks Blocks', 'Blocks Sh', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Total Cmp', 'Total Att', 'Total Cmp%', 'Total TotDist', 'Total PrgDist', 'Short Cmp', 'Short Att', 'Short Cmp%', 'Medium Cmp', 'Medium Att', 'Medium Cmp%', 'Long Cmp', 'Long Att', 'Long Cmp%', 'Ast', 'xAG', 'xA', 'KP', '3-Jan', 'PPA', 'CrsPA', 'PrgP', 'SCA SCA', 'SCA GCA', 

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
season_dir = "/Users/home/Capstone/ADS599_Capstone/Main Notebook/Data Folder/DataExtracted/"
rebalanced_scores_path = "/Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv"

season_files = [
    "real_madrid_schedule_17_18.csv",
    "real_madrid_schedule_18_19.csv",
    "real_madrid_schedule_19_20.csv",
    "real_madrid_schedule_20_21.csv",
    "real_madrid_schedule_21_22.csv",
    "real_madrid_schedule_22_23.csv",
    "real_madrid_schedule_23_24 (1).csv",
    "real_madrid_schedule_24_25 (1).csv"
]

def load_and_combine_schedules():
    """
    Load and combine all schedule files
    """
    print("Loading schedule files...")
    
    all_schedules = []
    
    for file in season_files:
        file_path = os.path.join(season_dir, file)
        
        if os.path.exists(file_path):
            print(f"✓ Loading: {file}")
            df = pd.read_csv(file_path)
            
            # Add season info
            season = file.split('_')[-1].replace('.csv', '').replace(' (1)', '')
            df['Season'] = season
            
            all_schedules.append(df)
        else:
            print(f"❌ File not found: {file}")
    
    # Combine all schedules
    combined_schedule = pd.concat(all_schedules, ignore_index=True)
    print(f"Combined schedule shape: {combined_schedule.shape}")
    print(f"Columns: {list(combined_schedule.columns)}")
    
    return combined_schedule

def load_rebalanced_scores():
    """
    Load rebalanced scores data
    """
    print(f"\nLoading rebalanced scores from: {rebalanced_scores_path}")
    
    if os.path.exists(rebalanced_scores_path):
        scores_df = pd.read_csv(rebalanced_scores_path)
        print(f"✓ Rebalanced scores shape: {scores_df.shape}")
        print(f"Columns: {list(scores_df.columns)}")
        return scores_df
    else:
        print(f"❌ Rebalanced scores file not found!")
        return None

def create_win_loss_dataset():
    """
    Create dataset with Date, Win/Loss, and TEAM-LEVEL Rebalanced_Score
    """
    print("="*60)
    print("CREATING TEAM-LEVEL WIN/LOSS DATASET FOR LOGISTIC REGRESSION")
    print("="*60)
    
    # Load schedule data
    schedule_df = load_and_combine_schedules()
    
    # Load rebalanced scores
    scores_df = load_rebalanced_scores()
    
    if scores_df is None:
        print("❌ Cannot proceed without rebalanced scores")
        return None
    
    print(f"\nSchedule data preview:")
    print(schedule_df.head())
    
    print(f"\nRebalanced scores preview:")
    print(scores_df.head())
    
    # Clean and standardize date formats
    print("\nProcessing dates...")
    
    # For schedule data - try to find date column
    date_cols = [col for col in schedule_df.columns if 'date' in col.lower()]
    if date_cols:
        schedule_df['Date'] = pd.to_datetime(schedule_df[date_cols[0]], errors='coerce')
    else:
        print("Available schedule columns:", list(schedule_df.columns))
        date_col = input("Enter the name of the DATE column in schedule data: ")
        schedule_df['Date'] = pd.to_datetime(schedule_df[date_col], errors='coerce')
    
    # For scores data - try to find date column
    date_cols_scores = [col for col in scores_df.columns if 'date' in col.lower()]
    if date_cols_scores:
        scores_df['Date'] = pd.to_datetime(scores_df[date_cols_scores[0]], errors='coerce')
    else:
        print("Available rebalanced scores columns:", list(scores_df.columns))
        date_col = input("Enter the name of the DATE column in rebalanced scores: ")
        scores_df['Date'] = pd.to_datetime(scores_df[date_col], errors='coerce')
    
    # Find result column in schedule
    result_cols = [col for col in schedule_df.columns if any(word in col.lower() for word in ['result', 'outcome'])]
    if result_cols:
        result_col = result_cols[0]
    else:
        print("Available schedule columns:", list(schedule_df.columns))
        result_col = input("Enter the name of the RESULT column (W/L/D): ")
    
    # Find rebalanced score column
    score_cols = [col for col in scores_df.columns if 'rebalanced' in col.lower() or 'score' in col.lower()]
    if score_cols:
        score_col = score_cols[0]
    else:
        print("Available rebalanced scores columns:", list(scores_df.columns))
        score_col = input("Enter the name of the REBALANCED SCORE column: ")
    
    print(f"Using result column: {result_col}")
    print(f"Using score column: {score_col}")
    
    # Create clean schedule dataframe
    schedule_clean = schedule_df[['Date', result_col]].copy()
    schedule_clean.columns = ['Date', 'Result']
    schedule_clean = schedule_clean.dropna()
    
    # AGGREGATE REBALANCED SCORES BY MATCH (TEAM LEVEL)
    print(f"\n📊 AGGREGATING TEAM PERFORMANCE BY MATCH...")
    
    # Group by date and sum/average the rebalanced scores for the entire team
    team_scores = scores_df.groupby('Date').agg({
        score_col: ['sum', 'mean', 'count']
    }).reset_index()
    
    # Flatten column names
    team_scores.columns = ['Date', 'Team_Rebalanced_Score_Sum', 'Team_Rebalanced_Score_Mean', 'Players_Count']
    
    print(f"Team aggregated data shape: {team_scores.shape}")
    print("Team scores preview:")
    print(team_scores.head())
    
    print(f"\nTeam score statistics:")
    print(f"Sum - Range: {team_scores['Team_Rebalanced_Score_Sum'].min():.2f} to {team_scores['Team_Rebalanced_Score_Sum'].max():.2f}")
    print(f"Mean - Range: {team_scores['Team_Rebalanced_Score_Mean'].min():.2f} to {team_scores['Team_Rebalanced_Score_Mean'].max():.2f}")
    print(f"Average players per match: {team_scores['Players_Count'].mean():.1f}")
    
    # Merge schedule with team-level scores
    print("\nMerging schedule with TEAM-LEVEL scores...")
    merged_df = pd.merge(schedule_clean, team_scores, on='Date', how='inner')
    
    print(f"Merged dataset shape: {merged_df.shape}")
    
    if merged_df.empty:
        print("❌ No matching dates found between schedule and team scores!")
        print("Schedule date range:", schedule_clean['Date'].min(), "to", schedule_clean['Date'].max())
        print("Team scores date range:", team_scores['Date'].min(), "to", team_scores['Date'].max())
        return None
    
    # Filter for Win/Loss only (remove draws)
    print("\nFiltering Win/Loss matches...")
    win_loss_df = merged_df[merged_df['Result'].isin(['W', 'L'])].copy()
    
    # Create binary target
    win_loss_df['Win'] = (win_loss_df['Result'] == 'W').astype(int)
    
    print(f"Final dataset shape: {win_loss_df.shape}")
    print("Result distribution:")
    print(win_loss_df['Result'].value_counts())
    print("Win distribution:")
    print(win_loss_df['Win'].value_counts())
    
    # Ask user which team score to use for analysis
    print(f"\nChoose which TEAM score to use for logistic regression:")
    print(f"1. Team_Rebalanced_Score_Sum (total team performance)")
    print(f"2. Team_Rebalanced_Score_Mean (average player performance)")
    
    choice = input("Enter choice (1 or 2): ").strip()
    
    if choice == "2":
        score_column = 'Team_Rebalanced_Score_Mean'
        print("✓ Using MEAN team score (average player performance)")
    else:
        score_column = 'Team_Rebalanced_Score_Sum'
        print("✓ Using SUM team score (total team performance)")
    
    # Final clean dataset with chosen score
    final_df = win_loss_df[['Date', 'Result', score_column, 'Players_Count', 'Win']].copy()
    final_df.rename(columns={score_column: 'Rebalanced_Score'}, inplace=True)
    
    print(f"\nFinal team-level dataset:")
    print(final_df.head(10))
    
    return final_df

def create_train_test_splits(df):
    """
    Create training and test splits
    """
    print("\n" + "="*40)
    print("CREATING TRAIN/TEST SPLITS")
    print("="*40)
    
    # Features and target
    X = df[['Rebalanced_Score']].copy()
    y = df['Win'].copy()
    
    print(f"Features shape: {X.shape}")
    print(f"Target shape: {y.shape}")
    
    # Split data: 70% train, 30% test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"Training set: {X_train.shape[0]} matches")
    print(f"Test set: {X_test.shape[0]} matches")
    
    return X_train, X_test, y_train, y_test

def plot_logistic_curve(df, model, scaler):
    """
    Plot the S-curve showing how Rebalanced_Score predicts Win probability
    """
    print("\n" + "="*40)
    print("PLOTTING LOGISTIC S-CURVE")
    print("="*40)
    
    # First, let's analyze the data distribution
    print("DATA ANALYSIS:")
    print(f"Rebalanced Score range: {df['Rebalanced_Score'].min():.2f} to {df['Rebalanced_Score'].max():.2f}")
    print(f"Mean: {df['Rebalanced_Score'].mean():.2f}, Std: {df['Rebalanced_Score'].std():.2f}")
    print(f"Win rate: {df['Win'].mean():.2%}")
    
    # Check if we have enough variation
    if df['Rebalanced_Score'].std() < 1:
        print("⚠️  WARNING: Low variation in Rebalanced_Score - S-curve may be flat")
    
    # Create WIDER range of Rebalanced_Score values for plotting
    score_min = df['Rebalanced_Score'].min() - df['Rebalanced_Score'].std()
    score_max = df['Rebalanced_Score'].max() + df['Rebalanced_Score'].std()
    score_range = np.linspace(score_min, score_max, 200).reshape(-1, 1)
    
    # Scale the range using the same scaler
    score_range_scaled = scaler.transform(score_range)
    
    # Predict probabilities
    win_probabilities = model.predict_proba(score_range_scaled)[:, 1]
    
    print(f"Predicted probability range: {win_probabilities.min():.3f} to {win_probabilities.max():.3f}")
    
    # Create the plot
    plt.figure(figsize=(15, 10))
    
    # Plot the S-curve with better scaling
    plt.subplot(2, 3, 1)
    plt.plot(score_range, win_probabilities, 'b-', linewidth=3, label='Logistic Curve')
    plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='50% Probability')
    
    # Add actual data points with jitter for better visibility
    wins = df[df['Win'] == 1]['Rebalanced_Score']
    losses = df[df['Win'] == 0]['Rebalanced_Score']
    
    # Add some jitter to y-values to see overlapping points
    win_jitter = np.random.normal(1, 0.02, len(wins))
    loss_jitter = np.random.normal(0, 0.02, len(losses))
    
    plt.scatter(wins, win_jitter, color='green', alpha=0.6, s=30, label=f'Wins ({len(wins)})')
    plt.scatter(losses, loss_jitter, color='red', alpha=0.6, s=30, label=f'Losses ({len(losses)})')
    
    plt.xlabel('Rebalanced Score')
    plt.ylabel('Win Probability')
    plt.title('S-Curve: Rebalanced Score vs Win Probability')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(-0.1, 1.1)
    
    # Distribution of Rebalanced Scores by result
    plt.subplot(2, 3, 2)
    plt.hist(losses, bins=15, alpha=0.7, color='red', label=f'Losses ({len(losses)})', density=True)
    plt.hist(wins, bins=15, alpha=0.7, color='green', label=f'Wins ({len(wins)})', density=True)
    plt.xlabel('Rebalanced Score')
    plt.ylabel('Density')
    plt.title('Distribution of Rebalanced Scores')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Box plot comparison
    plt.subplot(2, 3, 3)
    box_data = [losses, wins]
    box_plot = plt.boxplot(box_data, labels=['Losses', 'Wins'], patch_artist=True)
    box_plot['boxes'][0].set_facecolor('red')
    box_plot['boxes'][1].set_facecolor('green')
    plt.ylabel('Rebalanced Score')
    plt.title('Rebalanced Score by Result')
    plt.grid(True, alpha=0.3)
    
    # Show the logistic function equation
    plt.subplot(2, 3, 4)
    plt.text(0.1, 0.8, f"Logistic Regression Equation:", fontsize=12, fontweight='bold')
    plt.text(0.1, 0.6, f"P(Win) = 1 / (1 + e^-(β₀ + β₁×Score))", fontsize=11)
    plt.text(0.1, 0.4, f"Where:", fontsize=11)
    plt.text(0.1, 0.3, f"β₀ (intercept) = {model.intercept_[0]:.4f}", fontsize=10)
    plt.text(0.1, 0.2, f"β₁ (coefficient) = {model.coef_[0][0]:.4f}", fontsize=10)
    plt.text(0.1, 0.05, f"Interpretation: Each 1-unit increase in\nRebalanced Score multiplies odds by {np.exp(model.coef_[0][0]):.3f}", fontsize=9)
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.axis('off')
    plt.title('Model Parameters')
    
    # Probability bins analysis
    plt.subplot(2, 3, 5)
    
    # Create probability predictions for actual data
    X_actual = df[['Rebalanced_Score']].values
    X_actual_scaled = scaler.transform(X_actual)
    y_pred_proba_actual = model.predict_proba(X_actual_scaled)[:, 1]
    
    # Create bins and calculate actual win rate
    bins = np.linspace(0, 1, 11)  # 10 bins from 0 to 1
    
    actual_win_rates = []
    predicted_probs = []
    
    for i in range(len(bins)-1):
        mask = (y_pred_proba_actual >= bins[i]) & (y_pred_proba_actual < bins[i+1])
        if mask.sum() > 0:
            actual_rate = df.loc[mask, 'Win'].mean()
            predicted_prob = y_pred_proba_actual[mask].mean()
            actual_win_rates.append(actual_rate)
            predicted_probs.append(predicted_prob)
    
    # Plot calibration
    plt.plot([0, 1], [0, 1], 'k--', alpha=0.7, label='Perfect Calibration')
    if predicted_probs:
        plt.plot(predicted_probs, actual_win_rates, 'bo-', markersize=8, label='Model Calibration')
    plt.xlabel('Predicted Win Probability')
    plt.ylabel('Actual Win Rate')
    plt.title('Model Calibration')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Score vs Probability scatter
    plt.subplot(2, 3, 6)
    colors = ['red' if w == 0 else 'green' for w in df['Win']]
    plt.scatter(df['Rebalanced_Score'], y_pred_proba_actual, c=colors, alpha=0.6)
    plt.xlabel('Rebalanced Score')
    plt.ylabel('Predicted Win Probability')
    plt.title('Score vs Predicted Probability')
    plt.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(df['Rebalanced_Score'], y_pred_proba_actual, 1)
    p = np.poly1d(z)
    plt.plot(df['Rebalanced_Score'], p(df['Rebalanced_Score']), "b--", alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed insights
    print(f"\nDETAILED ANALYSIS:")
    print(f"─" * 50)
    
    # Statistical significance
    wins_scores = df[df['Win'] == 1]['Rebalanced_Score']
    losses_scores = df[df['Win'] == 0]['Rebalanced_Score']
    
    from scipy import stats
    t_stat, p_value = stats.ttest_ind(wins_scores, losses_scores)
    
    print(f"📊 Win scores mean: {wins_scores.mean():.2f} ± {wins_scores.std():.2f}")
    print(f"📊 Loss scores mean: {losses_scores.mean():.2f} ± {losses_scores.std():.2f}")
    print(f"📊 Difference: {wins_scores.mean() - losses_scores.mean():.2f}")
    print(f"📊 T-test p-value: {p_value:.4f} ({'Significant' if p_value < 0.05 else 'Not significant'})")
    
    # Find score thresholds
    threshold_50 = None
    for score, prob in zip(score_range.flatten(), win_probabilities):
        if prob >= 0.5 and threshold_50 is None:
            threshold_50 = score
            break
    
    if threshold_50:
        print(f"🎯 Rebalanced Score ≥ {threshold_50:.2f} → Win probability ≥ 50%")
    
    # Score ranges analysis
    q33 = df['Rebalanced_Score'].quantile(0.33)
    q67 = df['Rebalanced_Score'].quantile(0.67)
    
    low_scores = df[df['Rebalanced_Score'] < q33]
    mid_scores = df[(df['Rebalanced_Score'] >= q33) & (df['Rebalanced_Score'] < q67)]
    high_scores = df[df['Rebalanced_Score'] >= q67]
    
    print(f"🔻 Low scores (< {q33:.1f}): {low_scores['Win'].mean():.1%} win rate ({len(low_scores)} matches)")
    print(f"➖ Mid scores ({q33:.1f}-{q67:.1f}): {mid_scores['Win'].mean():.1%} win rate ({len(mid_scores)} matches)") 
    print(f"🔺 High scores (> {q67:.1f}): {high_scores['Win'].mean():.1%} win rate ({len(high_scores)} matches)")
    
    # Model strength assessment
    prob_range = win_probabilities.max() - win_probabilities.min()
    if prob_range < 0.3:
        print(f"⚠️  Model shows WEAK discrimination (probability range: {prob_range:.3f})")
    elif prob_range < 0.6:
        print(f"📈 Model shows MODERATE discrimination (probability range: {prob_range:.3f})")
    else:
        print(f"🚀 Model shows STRONG discrimination (probability range: {prob_range:.3f})")

def run_logistic_regression(X_train, X_test, y_train, y_test):
    """
    Run logistic regression and show results
    """
    print("\n" + "="*40)
    print("LOGISTIC REGRESSION RESULTS")
    print("="*40)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit model
    lr_model = LogisticRegression(random_state=42)
    lr_model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = lr_model.predict(X_test_scaled)
    y_pred_proba = lr_model.predict_proba(X_test_scaled)[:, 1]
    
    # Results
    train_acc = lr_model.score(X_train_scaled, y_train)
    test_acc = lr_model.score(X_test_scaled, y_test)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    print(f"Training Accuracy: {train_acc:.3f}")
    print(f"Test Accuracy: {test_acc:.3f}")
    print(f"ROC AUC Score: {auc_score:.3f}")
    
    print(f"\nRebalanced Score Coefficient: {lr_model.coef_[0][0]:.4f}")
    print(f"Intercept: {lr_model.intercept_[0]:.4f}")
    
    if lr_model.coef_[0][0] > 0:
        print("✓ Higher Rebalanced Score increases WIN probability")
    else:
        print("✗ Higher Rebalanced Score decreases WIN probability")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return lr_model, scaler

def main():
    """
    Main execution function
    """
    # Create the dataset
    final_df = create_win_loss_dataset()
    
    if final_df is None:
        print("❌ Failed to create dataset")
        return
    
    print(f"\n✓ Successfully created dataset with {len(final_df)} matches")
    print("\nDataset preview:")
    print(final_df.head(10))
    
    # Save the dataset
    output_path = "/Users/mariamoramora/Documents/GitHub/ADS599_Capstone/Main Notebook/Data Folder/win_loss_rebalanced_dataset.csv"
    final_df.to_csv(output_path, index=False)
    print(f"\n✓ Dataset saved to: {output_path}")
    
    # Create train/test splits
    X_train, X_test, y_train, y_test = create_train_test_splits(final_df)
    
    # Run logistic regression
    model, scaler = run_logistic_regression(X_train, X_test, y_train, y_test)
    
    # Plot the S-curve and analysis
    plot_logistic_curve(final_df, model, scaler)
    
    print("\n" + "="*60)
    print("SUCCESS! Ready for logistic regression analysis")
    print("="*60)
    
    return final_df, model, scaler

# Run the main function
if __name__ == "__main__":
    final_df, model, scaler = main()

/var/folders/q2/5yylt10156q8bq2drpk08r7w0000gn/T/ipykernel_54019/1983041645.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule_df['Date'] = pd.to_datetime(schedule_df[date_cols[0]], errors='coerce')


CREATING TEAM-LEVEL WIN/LOSS DATASET FOR LOGISTIC REGRESSION
Loading schedule files...
✓ Loading: real_madrid_schedule_17_18.csv
✓ Loading: real_madrid_schedule_18_19.csv
✓ Loading: real_madrid_schedule_19_20.csv
✓ Loading: real_madrid_schedule_20_21.csv
✓ Loading: real_madrid_schedule_21_22.csv
✓ Loading: real_madrid_schedule_22_23.csv
✓ Loading: real_madrid_schedule_23_24 (1).csv
✓ Loading: real_madrid_schedule_24_25 (1).csv
Combined schedule shape: (418, 21)
Columns: ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation', 'Opp Formation', 'Referee', 'Match Report', 'Notes', 'Season']

Loading rebalanced scores from: /Users/home/Capstone/ADS599_Capstone/Main Notebook/Code Library Folder/02_Feature_Engineering/outputs/real_madrid_rebalanced_scores.csv
✓ Rebalanced scores shape: (7140, 80)
Columns: ['Date', 'Competition', 'Opponent', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min', ' Gls', ' Ast', ' PK'

In [ ]:
def load_and_combine_schedules():
    """Load and combine all schedule files"""
    print("Loading schedule files...")
    all_schedules = []
    
    for file in season_files:
        file_path = os.path.join(season_dir, file)
        if os.path.exists(file_path):
            print(f"✓ Loading: {file}")
            df = pd.read_csv(file_path)
            season = file.split('_')[-1].replace('.csv', '').replace(' (1)', '')
            df['Season'] = season
            all_schedules.append(df)
        else:
            print(f"❌ File not found: {file}")
    
    combined_schedule = pd.concat(all_schedules, ignore_index=True)
    print(f"Combined schedule shape: {combined_schedule.shape}")
    return combined_schedule

def load_rebalanced_scores():
    """Load rebalanced scores data"""
    print(f"\nLoading rebalanced scores from: {rebalanced_scores_path}")
    
    if os.path.exists(rebalanced_scores_path):
        scores_df = pd.read_csv(rebalanced_scores_path)
        print(f"✓ Rebalanced scores shape: {scores_df.shape}")
        print(f"Columns: {list(scores_df.columns)}")
        return scores_df
    else:
        print(f"❌ Rebalanced scores file not found!")
        return None

def classify_position(pos):
    """Classify position into Forward, Midfielder, Defender"""
    if pd.isna(pos):
        return 'Unknown'
    
    pos_str = str(pos).upper()
    
    # Forward positions
    if any(keyword in pos_str for keyword in ['FW', 'CF', 'LW', 'RW', 'ST', 'FORWARD']):
        return 'Forward'
    # Midfielder positions  
    elif any(keyword in pos_str for keyword in ['MF', 'CM', 'DM', 'AM', 'LM', 'RM', 'CDM', 'CAM', 'MIDFIELDER']):
        return 'Midfielder'
    # Defender positions
    elif any(keyword in pos_str for keyword in ['DF', 'CB', 'LB', 'RB', 'WB', 'SW', 'DEFENDER']):
        return 'Defender'
    # Goalkeeper
    elif any(keyword in pos_str for keyword in ['GK', 'GOALKEEPER']):
        return 'Goalkeeper'
    else:
        return 'Unknown'

def create_position_based_dataset():
    """Create dataset with position-based analysis including multiple features"""
    print("="*60)
    print("CREATING POSITION-BASED WIN/LOSS DATASET WITH MULTIPLE FEATURES")
    print("="*60)
    
    # Load data
    schedule_df = load_and_combine_schedules()
    scores_df = load_rebalanced_scores()
    
    if scores_df is None:
        return None
    
    print(f"\nAvailable columns in rebalanced scores:")
    print(scores_df.columns.tolist())
    
    # Process dates
    print("\nProcessing dates...")
    date_cols = [col for col in schedule_df.columns if 'date' in col.lower()]
    if date_cols:
        schedule_df['Date'] = pd.to_datetime(schedule_df[date_cols[0]], errors='coerce')
    else:
        print("Available schedule columns:", list(schedule_df.columns))
        date_col = input("Enter the name of the DATE column in schedule data: ")
        schedule_df['Date'] = pd.to_datetime(schedule_df[date_col], errors='coerce')
    
    date_cols_scores = [col for col in scores_df.columns if 'date' in col.lower()]
    if date_cols_scores:
        scores_df['Date'] = pd.to_datetime(scores_df[date_cols_scores[0]], errors='coerce')
    else:
        print("Available rebalanced scores columns:", list(scores_df.columns))
        date_col = input("Enter the name of the DATE column in rebalanced scores: ")
        scores_df['Date'] = pd.to_datetime(scores_df[date_col], errors='coerce')
    
    # Find columns
    result_cols = [col for col in schedule_df.columns if any(word in col.lower() for word in ['result', 'outcome'])]
    result_col = result_cols[0] if result_cols else input("Enter the name of the RESULT column: ")
    
    pos_cols = [col for col in scores_df.columns if any(word in col.lower() for word in ['pos', 'position'])]
    position_col = pos_cols[0] if pos_cols else input("Enter the name of the POSITION column: ")
    
    print(f"Using: {result_col}, {position_col}")
    
    # Clean schedule data
    schedule_clean = schedule_df[['Date', result_col]].copy()
    schedule_clean.columns = ['Date', 'Result']
    schedule_clean = schedule_clean.dropna()
    
    # Classify positions
    print(f"\n📊 CLASSIFYING POSITIONS...")
    scores_df['Position_Group'] = scores_df[position_col].apply(classify_position)
    
    print("Position Group Classification:")
    print(scores_df['Position_Group'].value_counts())
    
    # Define features to aggregate by position based on your rebalanced score formula
    forward_features = [
        'Gls', 'Goals',  # Goals
        'Ast', 'Assists',  # Assists  
        'SoT', 'Shots_On_Target',  # Shots on Target
        'xG', 'Expected_Goals',  # Expected Goals
        'xAG', 'Expected_Assisted_Goals',  # Expected Assisted Goals
        'TakeOns', 'Take_Ons_Succ', 'Takeouts_Successful',  # Take-ons
        'Rebalanced_Score'  # Original rebalanced score
    ]
    
    midfielder_features = [
        'Passes_Cmp%', 'Pass_Completion_Percent', 'Total_Pass_Completion_Percent',  # Pass Completion %
        'KP', 'Key_Passes',  # Key Passes
        'Tkl', 'Tackles', 'Tackles_Made',  # Tackles
        'Carries_PrgC', 'Progressive_Carries',  # Progressive Carries
        'Passes_PrgP', 'Progressive_Passes',  # Progressive Passes
        'Touches',  # Touches
        'Rebalanced_Score'  # Original rebalanced score
    ]
    
    defender_features = [
        'Int', 'Interceptions',  # Interceptions
        'Blocks', 'Blocks_Made',  # Blocks
        'Clr', 'Clearances',  # Clearances
        'Tkl_W', 'Tackles_Won',  # Tackles Won
        'Tkl_Def', 'Tackles_Defensive_Third',  # Tackles Defensive 3rd
        'Tkl_Mid', 'Tackles_Middle_Third',  # Tackles Middle 3rd
        'Rebalanced_Score'  # Original rebalanced score
    ]
    
    goalkeeper_features = [
        'Total_Cmp%', 'Total_Pass_Completion_Percent',  # Total Pass Completion %
        'Err', 'Errors',  # Errors
        'Prg_Dist', 'Total_Progressive_Distance',  # Progressive Distance
        'Short_Cmp%', 'Short_Pass_Completion_Percent',  # Short Pass Completion %
        'Med_Cmp%', 'Medium_Pass_Completion_Percent',  # Medium Pass Completion %
        'Total_Cmp', 'Total_Passes_Completed',  # Total Passes Completed
        'Rebalanced_Score'  # Original rebalanced score
    ]
    
    # Find which features actually exist in the dataset
    available_columns = scores_df.columns.tolist()
    print(f"\n📊 IDENTIFYING AVAILABLE FEATURES...")
    
    def find_available_features(feature_list, feature_type):
        found_features = []
        for feature in feature_list:
            # Try exact match first, then partial matches
            if feature in available_columns:
                found_features.append(feature)
            else:
                # Look for similar column names
                matches = [col for col in available_columns if any(part in col for part in feature.split('_')) and len(col) > 2]
                if matches:
                    found_features.extend(matches[:1])  # Take first match
        
        # Remove duplicates while preserving order
        unique_features = []
        for f in found_features:
            if f not in unique_features:
                unique_features.append(f)
        
        print(f"{feature_type}: {len(unique_features)} features found - {unique_features}")
        return unique_features
    
    forward_available = find_available_features(forward_features, "FORWARD")
    midfielder_available = find_available_features(midfielder_features, "MIDFIELDER") 
    defender_available = find_available_features(defender_features, "DEFENDER")
    goalkeeper_available = find_available_features(goalkeeper_features, "GOALKEEPER")
    
    # Aggregate by position group per match with multiple features
    print(f"\n📊 AGGREGATING MULTIPLE FEATURES BY POSITION GROUP PER MATCH...")
    
    # Create aggregation dictionary for all features
    agg_dict = {}
    
    # Add features for each position if they exist
    all_features = list(set(forward_available + midfielder_available + defender_available + goalkeeper_available))
    
    for feature in all_features:
        if feature in available_columns:
            agg_dict[feature] = ['sum', 'mean', 'count']
    
    print(f"Aggregating {len(agg_dict)} features: {list(agg_dict.keys())}")
    
    # Group by date and position, aggregate all features
    position_stats = scores_df.groupby(['Date', 'Position_Group']).agg(agg_dict).reset_index()
    
    # Flatten column names
    new_columns = ['Date', 'Position_Group']
    for feature in agg_dict.keys():
        new_columns.extend([f'{feature}_sum', f'{feature}_mean', f'{feature}_count'])
    
    position_stats.columns = new_columns
    
    # Create separate dataframes for each position
    position_dfs = {}
    
    for position in ['Forward', 'Midfielder', 'Defender', 'Goalkeeper']:
        pos_data = position_stats[position_stats['Position_Group'] == position].copy()
        
        if len(pos_data) > 0:
            # Select relevant features for this position
            if position == 'Forward':
                relevant_features = [f for f in forward_available if f in agg_dict.keys()]
            elif position == 'Midfielder':
                relevant_features = [f for f in midfielder_available if f in agg_dict.keys()]
            elif position == 'Defender':
                relevant_features = [f for f in defender_available if f in agg_dict.keys()]
            else:  # Goalkeeper
                relevant_features = [f for f in goalkeeper_available if f in agg_dict.keys()]
            
            # Keep Date and relevant feature columns (sum and mean)
            keep_columns = ['Date']
            for feature in relevant_features:
                keep_columns.extend([f'{feature}_sum', f'{feature}_mean'])
            
            # Filter columns that actually exist
            existing_columns = [col for col in keep_columns if col in pos_data.columns]
            pos_subset = pos_data[existing_columns].copy()
            
            # Rename columns to include position prefix
            rename_dict = {}
            for col in pos_subset.columns:
                if col != 'Date':
                    rename_dict[col] = f'{position}_{col}'
            
            pos_subset.rename(columns=rename_dict, inplace=True)
            position_dfs[position] = pos_subset
            
            print(f"{position}: {len(pos_subset)} matches, {len(existing_columns)-1} features")
    
    # Merge all position dataframes
    merged_positions = None
    for position, pos_df in position_dfs.items():
        if merged_positions is None:
            merged_positions = pos_df.copy()
        else:
            merged_positions = pd.merge(merged_positions, pos_df, on='Date', how='outer')
    
    # Fill NaN values with 0 (for matches where position didn't play)
    merged_positions = merged_positions.fillna(0)
    
    print(f"Combined position data shape: {merged_positions.shape}")
    
    # Merge with schedule
    merged_df = pd.merge(schedule_clean, merged_positions, on='Date', how='inner')
    
    # Filter Win/Loss only
    win_loss_df = merged_df[merged_df['Result'].isin(['W', 'L'])].copy()
    win_loss_df['Win'] = (win_loss_df['Result'] == 'W').astype(int)
    
    print(f"Final dataset shape: {win_loss_df.shape}")
    print(f"Total features: {len(win_loss_df.columns) - 3}")  # Subtract Date, Result, Win
    
    # Show available features by position
    for position in ['Forward', 'Midfielder', 'Defender', 'Goalkeeper']:
        pos_columns = [col for col in win_loss_df.columns if col.startswith(f'{position}_')]
        if pos_columns:
            print(f"{position} features ({len(pos_columns)}): {pos_columns[:5]}..." if len(pos_columns) > 5 else f"{position} features: {pos_columns}")
    
    return win_loss_df

def plot_forward_analysis_only(df, score_col, model, scaler, position_name):
    """Plot S-curve, ROC curve, and confusion matrix for Forward analysis"""
    
    # Create EXTENDED range to show complete S-curve (0 to 1)
    data_std = df[score_col].std()
    data_mean = df[score_col].mean()
    
    # Extend range much further to ensure we reach 0% and 100% probability
    score_min = data_mean - 4 * data_std
    score_max = data_mean + 4 * data_std
    
    score_range = np.linspace(score_min, score_max, 500).reshape(-1, 1)
    score_range_scaled = scaler.transform(score_range)
    win_probabilities = model.predict_proba(score_range_scaled)[:, 1]
    
    print(f"{position_name} - Extended range: {score_min:.2f} to {score_max:.2f}")
    print(f"{position_name} - Probability range: {win_probabilities.min():.3f} to {win_probabilities.max():.3f}")
    
    # Create the plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Plot 1: COMPLETE S-CURVE
    ax1 = axes[0, 0]
    ax1.plot(score_range, win_probabilities, 'b-', linewidth=4, label='Complete S-Curve')
    ax1.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='50% Probability')
    ax1.axhline(y=0.1, color='orange', linestyle=':', alpha=0.5, label='10%')
    ax1.axhline(y=0.9, color='orange', linestyle=':', alpha=0.5, label='90%')
    
    # Add actual data points  
    wins = df[df['Win'] == 1][score_col]
    losses = df[df['Win'] == 0][score_col]
    
    win_jitter = np.random.normal(1, 0.02, len(wins))
    loss_jitter = np.random.normal(0, 0.02, len(losses))
    
    ax1.scatter(wins, win_jitter, color='green', alpha=0.7, s=50, label=f'Wins ({len(wins)})', zorder=5)
    ax1.scatter(losses, loss_jitter, color='red', alpha=0.7, s=50, label=f'Losses ({len(losses)})', zorder=5)
    
    # Highlight actual data range
    ax1.axvspan(df[score_col].min(), df[score_col].max(), alpha=0.15, color='yellow', label='Data Range')
    
    ax1.set_xlabel(f'{position_name} Rebalanced Score')
    ax1.set_ylabel('Win Probability')
    ax1.set_title(f'{position_name} - COMPLETE S-Curve (0% to 100%)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.set_ylim(-0.05, 1.05)
    
    # Plot 2: Distribution comparison
    ax2 = axes[0, 1]
    ax2.hist(losses, bins=15, alpha=0.7, color='red', label=f'Losses ({len(losses)})', density=True)
    ax2.hist(wins, bins=15, alpha=0.7, color='green', label=f'Wins ({len(wins)})', density=True)
    ax2.set_xlabel(f'{position_name} Rebalanced Score')
    ax2.set_ylabel('Density')
    ax2.set_title(f'{position_name} - Score Distribution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Add statistics
    ax2.text(0.05, 0.95, f'Win Mean: {wins.mean():.1f}\nLoss Mean: {losses.mean():.1f}\nDiff: {wins.mean()-losses.mean():.1f}', 
             transform=ax2.transAxes, verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Plot 3: Box plot
    ax3 = axes[0, 2]
    box_data = [losses, wins]
    box_plot = ax3.boxplot(box_data, labels=['Losses', 'Wins'], patch_artist=True)
    box_plot['boxes'][0].set_facecolor('red')
    box_plot['boxes'][1].set_facecolor('green')
    ax3.set_ylabel(f'{position_name} Rebalanced Score')
    ax3.set_title(f'{position_name} - Score by Result')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: ROC CURVE
    from sklearn.metrics import roc_curve, auc
    
    # Get predictions for ROC curve
    X_test = df[[score_col]]
    X_test_scaled = scaler.transform(X_test)
    y_test = df['Win']
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    ax4 = axes[1, 0]
    ax4.plot(fpr, tpr, color='darkorange', lw=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    ax4.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    ax4.set_xlim([0.0, 1.0])
    ax4.set_ylim([0.0, 1.05])
    ax4.set_xlabel('False Positive Rate')
    ax4.set_ylabel('True Positive Rate')
    ax4.set_title('ROC Curve')
    ax4.legend(loc="lower right")
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: CONFUSION MATRIX
    y_pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    
    ax5 = axes[1, 1]
    im = ax5.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax5.figure.colorbar(im, ax=ax5)
    
    # Add text annotations
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax5.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black",
                    fontsize=14, fontweight='bold')
    
    ax5.set_ylabel('True Label')
    ax5.set_xlabel('Predicted Label')
    ax5.set_title('Confusion Matrix')
    ax5.set_xticks([0, 1])
    ax5.set_yticks([0, 1])
    ax5.set_xticklabels(['Loss', 'Win'])
    ax5.set_yticklabels(['Loss', 'Win'])
    
    # Add confusion matrix details
    tn, fp, fn, tp = cm.ravel()
    ax5.text(0.5, -0.15, f'True Negatives: {tn}\nFalse Positives: {fp}\nFalse Negatives: {fn}\nTrue Positives: {tp}', 
             transform=ax5.transAxes, ha='center',
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    # Plot 6: Performance Metrics
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    metrics_text = f"""
    PERFORMANCE METRICS
    ──────────────────────
    Accuracy: {accuracy:.3f}
    Precision: {precision:.3f}
    Recall (Sensitivity): {recall:.3f}
    Specificity: {specificity:.3f}
    F1-Score: {f1_score:.3f}
    AUC: {roc_auc:.3f}
    
    CONFUSION MATRIX BREAKDOWN
    ──────────────────────────
    True Positives (TP): {tp}
    True Negatives (TN): {tn}
    False Positives (FP): {fp}
    False Negatives (FN): {fn}
    
    Total Predictions: {tp + tn + fp + fn}
    """
    
    ax6.text(0.1, 0.9, metrics_text, transform=ax6.transAxes, fontsize=11,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed analysis
    print(f"\n{position_name} DETAILED ANALYSIS:")
    print(f"─" * 50)
    
    # Find key thresholds
    threshold_10 = threshold_50 = threshold_90 = None
    for score, prob in zip(score_range.flatten(), win_probabilities):
        if prob >= 0.1 and threshold_10 is None:
            threshold_10 = score
        if prob >= 0.5 and threshold_50 is None:
            threshold_50 = score
        if prob >= 0.9 and threshold_90 is None:
            threshold_90 = score
    
    print(f"📉 Score for 10% win prob: {threshold_10:.2f}")
    print(f"🎯 Score for 50% win prob: {threshold_50:.2f}")
    print(f"📈 Score for 90% win prob: {threshold_90:.2f}")
    
    # Statistical test
    t_stat, p_value = stats.ttest_ind(wins, losses)
    print(f"📊 T-test p-value: {p_value:.4f} ({'Significant' if p_value < 0.05 else 'Not significant'})")
    
    print(f"\n🎯 CLASSIFICATION RESULTS:")
    print(f"True Positives (Correctly predicted wins): {tp}")
    print(f"True Negatives (Correctly predicted losses): {tn}")
    print(f"False Positives (Predicted win, but lost): {fp}")
    print(f"False Negatives (Predicted loss, but won): {fn}")
    
    return {
        'position': position_name,
        'threshold_50': threshold_50,
        'win_mean': wins.mean(),
        'loss_mean': losses.mean(),
        'p_value': p_value,
        'coefficient': model.coef_[0][0],
        'auc': roc_auc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'confusion_matrix': {'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn}
    }

def analyze_all_positions_with_multiple_features(df):
    """Run analysis for Forward position only"""
    print("\n" + "="*60)
    print("ANALYZING FORWARD POSITION ONLY")
    print("="*60)
    
    results = []
    position = 'Forward'  # Only analyze Forward
    
    # Find all features for Forward position
    pos_columns = [col for col in df.columns if col.startswith(f'{position}_') and ('_sum' in col or '_mean' in col)]
    
    if not pos_columns:
        print(f"❌ No features found for {position}")
        return results
        
    print(f"\n{'='*50}")
    print(f"ANALYZING {position.upper()} ({len(pos_columns)} features)")
    print(f"{'='*50}")
    
    # Show available features
    sum_features = [col for col in pos_columns if '_sum' in col]
    mean_features = [col for col in pos_columns if '_mean' in col]
    
    print(f"📊 SUM features ({len(sum_features)}):")
    for feat in sum_features[:8]:  # Show first 8
        print(f"   • {feat}")
    if len(sum_features) > 8:
        print(f"   ... and {len(sum_features)-8} more")
        
    print(f"📊 MEAN features ({len(mean_features)}):")
    for feat in mean_features[:8]:  # Show first 8
        print(f"   • {feat}")
    if len(mean_features) > 8:
        print(f"   ... and {len(mean_features)-8} more")
    
    # Create dataset for Forward position
    analysis_columns = ['Date', 'Result', 'Win'] + pos_columns
    pos_data = df[analysis_columns].copy()
    pos_data = pos_data.dropna()
    
    if len(pos_data) < 20:
        print(f"⚠️ Skipping {position} - insufficient data ({len(pos_data)} matches)")
        return results
    
    print(f"📈 Data: {len(pos_data)} matches")
    
    # SINGLE FEATURE ANALYSIS (Rebalanced Score)
    rebalanced_cols = [col for col in pos_columns if 'Rebalanced_Score' in col and '_sum' in col]
    
    if rebalanced_cols:
        score_col = rebalanced_cols[0]
        print(f"\n🎯 SINGLE FEATURE ANALYSIS: {score_col}")
        
        X_single = pos_data[[score_col]]
        y = pos_data['Win']
        
        # Train single feature model
        X_train, X_test, y_train, y_test = train_test_split(X_single, y, test_size=0.3, random_state=42)
        
        scaler_single = StandardScaler()
        X_train_scaled = scaler_single.fit_transform(X_train)
        X_test_scaled = scaler_single.transform(X_test)
        
        model_single = LogisticRegression(random_state=42)
        model_single.fit(X_train_scaled, y_train)
        
        y_pred_proba_single = model_single.predict_proba(X_test_scaled)[:, 1]
        auc_single = roc_auc_score(y_test, y_pred_proba_single)
        
        print(f"   Single feature AUC: {auc_single:.3f}")
        
        # Plot Forward analysis with S-curve, ROC, and confusion matrix
        result_single = plot_forward_analysis_only(pos_data, score_col, model_single, scaler_single, f"{position} (Rebalanced Score Only)")
        result_single['model_type'] = 'Single Feature'
        results.append(result_single)
    
    # MULTIPLE FEATURES ANALYSIS
    print(f"\n🔥 MULTIPLE FEATURES ANALYSIS: All {len(pos_columns)} features")
    
    # Use all Forward features
    X_multi = pos_data[pos_columns]
    
    # Remove features with zero variance
    X_multi = X_multi.loc[:, X_multi.var() > 0.01]
    
    if X_multi.shape[1] < 2:
        print(f"⚠️ Insufficient features with variance for {position}")
        return results
        
    print(f"   Using {X_multi.shape[1]} features after variance filtering")
    
    # Train multiple features model
    X_train_multi, X_test_multi, y_train, y_test = train_test_split(X_multi, y, test_size=0.3, random_state=42)
    
    scaler_multi = StandardScaler()
    X_train_multi_scaled = scaler_multi.fit_transform(X_train_multi)
    X_test_multi_scaled = scaler_multi.transform(X_test_multi)
    
    model_multi = LogisticRegression(random_state=42, max_iter=1000)
    model_multi.fit(X_train_multi_scaled, y_train)
    
    y_pred_proba_multi = model_multi.predict_proba(X_test_multi_scaled)[:, 1]
    auc_multi = roc_auc_score(y_test, y_pred_proba_multi)
    
    print(f"   Multiple features AUC: {auc_multi:.3f}")
    
    # Show feature importance for multi-feature model
    feature_importance = pd.DataFrame({
        'Feature': X_multi.columns,
        'Coefficient': model_multi.coef_[0],
        'Abs_Coefficient': np.abs(model_multi.coef_[0])
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print(f"\n🏆 TOP 5 MOST IMPORTANT FEATURES:")
    for i, row in feature_importance.head(5).iterrows():
        direction = "↗️" if row['Coefficient'] > 0 else "↘️"
        print(f"   {i+1}. {row['Feature']}: {row['Coefficient']:+.3f} {direction}")
    
    # Get top feature for visualization
    top_feature = feature_importance.iloc[0]['Feature']
    
    print(f"\n📊 Top feature identified: {top_feature}")
    
    # Store results (no plotting)
    result_multi = {
        'position': position,
        'model_type': 'Multiple Features',
        'auc': auc_multi,
        'num_features': X_multi.shape[1],
        'top_feature': top_feature,
        'top_coefficient': feature_importance.iloc[0]['Coefficient'],
        'feature_importance': feature_importance.head(10).to_dict('records')
    }
    results.append(result_multi)
    
    # Compare single vs multiple
    if rebalanced_cols:
        improvement = auc_multi - auc_single
        print(f"\n📊 MODEL COMPARISON:")
        print(f"   Single Feature AUC: {auc_single:.3f}")
        print(f"   Multiple Features AUC: {auc_multi:.3f}")
        print(f"   Improvement: {improvement:+.3f} ({'Better' if improvement > 0.02 else 'Marginal' if improvement > 0 else 'Worse'})")
    
    # Final summary for Forward only
    print("\n" + "="*60)
    print("FORWARD POSITION ANALYSIS SUMMARY")
    print("="*60)
    
    if results:
        single_results = [r for r in results if r.get('model_type') == 'Single Feature']
        multi_results = [r for r in results if r.get('model_type') == 'Multiple Features']
        
        if single_results:
            print("\n🎯 SINGLE FEATURE MODEL (Rebalanced Score Only):")
            for r in single_results:
                print(f"   AUC: {r['auc']:.3f}")
        
        if multi_results:
            print(f"\n🔥 MULTIPLE FEATURES MODEL:")
            for r in multi_results:
                print(f"   AUC: {r['auc']:.3f} | Features: {r['num_features']} | Top: {r['top_feature'].split('_')[-1]}")
                
        if single_results and multi_results:
            improvement = multi_results[0]['auc'] - single_results[0]['auc']
            status = "🚀 Significant" if improvement > 0.02 else "📈 Marginal" if improvement > 0 else "📉 Worse"
            print(f"\n📈 MULTI-FEATURE BENEFIT: {improvement:+.3f} | {status}")
    
    return results

def main():
    """Main execution with comprehensive position-based analysis"""
    
    print("🚀 REAL MADRID COMPREHENSIVE POSITION ANALYSIS")
    print("="*60)
    print("📊 Features based on your rebalanced score formulas:")
    print("\n📊 DEFENSE FEATURES:")
    print("   • Interceptions (Int) - Coefficient: 2.5")
    print("   • Blocks - Coefficient: 2.0") 
    print("   • Clearances (Clr) - Coefficient: 1.0")
    print("   • Tackles Won (TklW) - Coefficient: 2.0")
    print("   • Defensive 3rd Tackles - Coefficient: 1.3")
    print("   • Middle 3rd Tackles - Coefficient: 0.8")
    
    print("\n📊 MIDFIELD FEATURES:")
    print("   • Pass Completion % - Coefficient: 2.5")
    print("   • Key Passes (KP) - Coefficient: 1.2")
    print("   • Tackles - Coefficient: 1.5")
    print("   • Progressive Carries - Coefficient: 0.8")
    print("   • Progressive Passes - Coefficient: 1.8")
    print("   • Touches - Coefficient: 0.3")
    
    print("\n📊 FORWARD FEATURES:")
    print("   • Goals (Gls) - Coefficient: 3.0")
    print("   • Assists (Ast) - Coefficient: 2.0")
    print("   • Shots on Target (SoT) - Coefficient: 1.0")
    print("   • Expected Goals (xG) - Coefficient: 1.5")
    print("   • Expected Assisted Goals (xAG) - Coefficient: 1.0")
    print("   • Take-ons - Coefficient: 0.5")
    
    print("\n📊 GOALKEEPER FEATURES:")
    print("   • Total Pass Completion % - Coefficient: 3.0")
    print("   • Errors - Coefficient: -2.0")
    print("   • Progressive Distance - Coefficient: 1.0")
    print("   • Short Pass Completion % - Coefficient: 1.5")
    print("   • Medium Pass Completion % - Coefficient: 1.0")
    print("   • Total Passes Completed - Coefficient: 0.5")
    print("="*60)
    
    # Create position-based dataset with multiple features
    df = create_position_based_dataset()
    
    if df is None:
        print("❌ Failed to create dataset")
        return
    
    print(f"\n✅ Successfully created dataset with {len(df)} matches")
    
    # Save dataset
    output_path = "/Users/mariamoramora/Documents/GitHub/ADS599_Capstone/Main Notebook/Data Folder/comprehensive_position_analysis.csv"
    df.to_csv(output_path, index=False)
    print(f"💾 Dataset saved to: {output_path}")
    
    print(f"\n📋 Dataset overview:")
    print(f"   • Total matches: {len(df)}")
    print(f"   • Wins: {df['Win'].sum()}")
    print(f"   • Losses: {len(df) - df['Win'].sum()}")
    print(f"   • Win rate: {df['Win'].mean():.1%}")
    print(f"   • Total features: {len(df.columns) - 3}")  # Subtract Date, Result, Win
    
    # Run comprehensive analysis
    results = analyze_all_positions_with_multiple_features(df)
    
    print("\n" + "="*60)
    print("✅ COMPREHENSIVE ANALYSIS COMPLETE!")
    print("="*60)
    print("🎯 Single feature analysis (Rebalanced Score only)")
    print("🔥 Multiple features analysis (All position-specific metrics)")
    print("📈 Complete S-curves (0% to 100%) for each position")
    print("🏆 Feature importance rankings")
    print("📊 Model comparison (Single vs Multiple features)")
    print("💾 All data saved for further analysis")
    print("="*60)
    
    return df, results

# Run the analysis
if __name__ == "__main__":
    df, results = main()

In [ ]:
# ==========================================
# EXTRACT AND PLOT ADJUSTED MODEL PERFORMANCE
# ==========================================

import matplotlib.pyplot as plt
import numpy as np

# Extract performance data from your adjusted models
def create_position_models_dict(adjusted_models):
    """Convert adjusted_models to the format expected by the plotting code"""
    position_models = {}
    
    for position, model_info in adjusted_models.items():
        # Create the expected structure
        position_models[position] = {
            'metrics': {
                'train_r2': 0.0,  # Not available in your data
                'test_r2': model_info['r2'],
                'train_mae': 0.0,  # Not available in your data
                'test_mae': model_info['mae']
            }
        }
    
    return position_models

# Convert your adjusted_models to the expected format
if 'adjusted_models' in locals() and adjusted_models:
    position_models = create_position_models_dict(adjusted_models)
    print("✅ Created position_models dictionary from adjusted_models")
    print(f"Available positions: {list(position_models.keys())}")
else:
    print("❌ adjusted_models not found. Please run the adjusted weighted scoring code first.")

# ==========================================
# PLOT ADJUSTED MODEL PERFORMANCE
# ==========================================

def plot_adjusted_model_performance(models_dict, title_prefix="Adjusted"):
    """Plot model performance with proper formatting"""
    
    if not models_dict:
        print("❌ No model data available for plotting")
        return
    
    print(f"📊 PLOTTING {title_prefix.upper()} MODEL PERFORMANCE BY POSITION")
    print("-" * 60)
    
    # Extract metrics
    positions = list(models_dict.keys())
    test_r2_scores = []
    test_mae_scores = []
    
    for pos in positions:
        test_r2_scores.append(models_dict[pos]['metrics']['test_r2'])
        test_mae_scores.append(models_dict[pos]['metrics']['test_mae'])
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: R² Scores by Position
    x = np.arange(len(positions))
    width = 0.6
    
    bars1 = ax1.bar(x, test_r2_scores, width, alpha=0.8, color='steelblue')
    
    ax1.set_xlabel('Position', fontweight='bold', fontsize=12)
    ax1.set_ylabel('R² Score', fontweight='bold', fontsize=12)
    ax1.set_title(f'{title_prefix} Model Performance - R² by Position', fontweight='bold', fontsize=14)
    ax1.set_xticks(x)
    ax1.set_xticklabels(positions, rotation=45, ha='right')
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.set_ylim(0, max(test_r2_scores) * 1.1)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=11, fontweight='bold')
    
    # Plot 2: MAE Scores by Position
    bars2 = ax2.bar(x, test_mae_scores, width, alpha=0.8, color='coral')
    
    ax2.set_xlabel('Position', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Mean Absolute Error', fontweight='bold', fontsize=12)
    ax2.set_title(f'{title_prefix} Model Performance - MAE by Position', fontweight='bold', fontsize=14)
    ax2.set_xticks(x)
    ax2.set_xticklabels(positions, rotation=45, ha='right')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar in bars2:
        height = bar.get_height()
        ax2.annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=11, fontweight='bold')
    
    plt.suptitle(f'{title_prefix} Model Performance Analysis by Position', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\n📊 {title_prefix.upper()} MODEL SUMMARY:")
    print("-" * 50)
    print(f"{'Position':<15} {'Test R²':<10} {'Test MAE':<10}")
    print("-" * 50)
    for i, pos in enumerate(positions):
        print(f"{pos:<15} {test_r2_scores[i]:<10.3f} {test_mae_scores[i]:<10.3f}")
    
    # Calculate averages
    avg_r2 = np.mean(test_r2_scores)
    avg_mae = np.mean(test_mae_scores)
    print("-" * 50)
    print(f"{'Average':<15} {avg_r2:<10.3f} {avg_mae:<10.3f}")
    
    return {
        'positions': positions,
        'test_r2': test_r2_scores,
        'test_mae': test_mae_scores,
        'avg_r2': avg_r2,
        'avg_mae': avg_mae
    }

# Plot the adjusted model performance
if 'position_models' in locals():
    adjusted_results = plot_adjusted_model_performance(position_models, "Adjusted Weighted Score")

# ==========================================
# COMPARISON WITH ORIGINAL (IF AVAILABLE)
# ==========================================

def compare_original_vs_adjusted(original_results, adjusted_results):
    """Compare original vs adjusted model performance"""
    
    if not original_results or not adjusted_results:
        print("❌ Need both original and adjusted results for comparison")
        return
    
    print(f"\n{'='*60}")
    print("COMPARISON: ORIGINAL vs ADJUSTED WEIGHTED SCORING")
    print(f"{'='*60}")
    
    # Find common positions
    original_positions = set(original_results['positions'])
    adjusted_positions = set(adjusted_results['positions'])
    common_positions = list(original_positions.intersection(adjusted_positions))
    
    if not common_positions:
        print("❌ No common positions found for comparison")
        return
    
    # Extract data for common positions
    orig_r2 = []
    adj_r2 = []
    orig_mae = []
    adj_mae = []
    
    for pos in common_positions:
        orig_idx = original_results['positions'].index(pos)
        adj_idx = adjusted_results['positions'].index(pos)
        
        orig_r2.append(original_results['test_r2'][orig_idx])
        adj_r2.append(adjusted_results['test_r2'][adj_idx])
        orig_mae.append(original_results['test_mae'][orig_idx])
        adj_mae.append(adjusted_results['test_mae'][adj_idx])
    
    # Create comparison plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    x = np.arange(len(common_positions))
    width = 0.35
    
    # R² Comparison
    bars1 = ax1.bar(x - width/2, orig_r2, width, label='Original', alpha=0.8, color='lightblue')
    bars2 = ax1.bar(x + width/2, adj_r2, width, label='Adjusted', alpha=0.8, color='darkblue')
    
    ax1.set_xlabel('Position', fontweight='bold')
    ax1.set_ylabel('Test R² Score', fontweight='bold')
    ax1.set_title('Model Comparison - Test R² by Position', fontweight='bold', fontsize=14)
    ax1.set_xticks(x)
    ax1.set_xticklabels(common_positions, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax1.annotate(f'{height:.3f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom',
                        fontsize=9)
    
    # MAE Comparison
    bars3 = ax2.bar(x - width/2, orig_mae, width, label='Original', alpha=0.8, color='lightcoral')
    bars4 = ax2.bar(x + width/2, adj_mae, width, label='Adjusted', alpha=0.8, color='darkred')
    
    ax2.set_xlabel('Position', fontweight='bold')
    ax2.set_ylabel('Test MAE', fontweight='bold')
    ax2.set_title('Model Comparison - Test MAE by Position', fontweight='bold', fontsize=14)
    ax2.set_xticks(x)
    ax2.set_xticklabels(common_positions, rotation=45, ha='right')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bars in [bars3, bars4]:
        for bar in bars:
            height = bar.get_height()
            ax2.annotate(f'{height:.3f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom',
                        fontsize=9)
    
    plt.suptitle('Original vs Adjusted Weighted Score Models', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Print comparison summary
    print(f"\n📊 COMPARISON SUMMARY:")
    print("-" * 70)
    print(f"{'Position':<15} {'Orig R²':<10} {'Adj R²':<10} {'Orig MAE':<10} {'Adj MAE':<10}")
    print("-" * 70)
    for i, pos in enumerate(common_positions):
        print(f"{pos:<15} {orig_r2[i]:<10.3f} {adj_r2[i]:<10.3f} {orig_mae[i]:<10.3f} {adj_mae[i]:<10.3f}")
    
    # Calculate improvements
    avg_orig_r2 = np.mean(orig_r2)
    avg_adj_r2 = np.mean(adj_r2)
    avg_orig_mae = np.mean(orig_mae)
    avg_adj_mae = np.mean(adj_mae)
    
    print("-" * 70)
    print(f"{'Average':<15} {avg_orig_r2:<10.3f} {avg_adj_r2:<10.3f} {avg_orig_mae:<10.3f} {avg_adj_mae:<10.3f}")
    
    # Calculate percentage improvements
    r2_improvement = ((avg_adj_r2 - avg_orig_r2) / avg_orig_r2) * 100
    mae_improvement = ((avg_orig_mae - avg_adj_mae) / avg_orig_mae) * 100
    
    print(f"\n🎯 Adjusted Model Performance vs Original:")
    print(f"   R² Score: {'↑' if r2_improvement > 0 else '↓'} {abs(r2_improvement):.1f}% {'improvement' if r2_improvement > 0 else 'decline'}")
    print(f"   MAE:      {'↓' if mae_improvement > 0 else '↑'} {abs(mae_improvement):.1f}% {'improvement' if mae_improvement > 0 else 'decline'}")

# ==========================================
# USAGE INSTRUCTIONS
# ==========================================

print(f"\n{'='*60}")
print("USAGE INSTRUCTIONS")
print(f"{'='*60}")

print("\n🔧 To use the original plotting code, you need:")
print("1. position_models dict with structure:")
print("   position_models['Position']['metrics']['test_r2']")
print("   position_models['Position']['metrics']['test_mae']")

print("\n2. For XGBoost comparison, you need:")
print("   xgboost_models['Position']['model_info']['metrics']['test_r2']")
print("   xgboost_models['Position']['model_info']['metrics']['val_r2']")

print("\n✅ Current status:")
if 'position_models' in locals():
    print(f"   position_models: Available with {len(position_models)} positions")
else:
    print("   position_models: Not available")

if 'adjusted_models' in locals():
    print(f"   adjusted_models: Available with {len(adjusted_models)} positions")
else:
    print("   adjusted_models: Not available")

print(f"\n🎯 Run this code after executing your adjusted weighted scoring analysis!")

In [ ]:
# ==========================================
# PLOT THE R2 and MAE for each position
# ==========================================    

import matplotlib.pyplot as plt
import numpy as np

# Check if we have the position models from RandomForest
if 'position_models' in locals() and position_models:
	print("📊 PLOTTING RANDOM FOREST MODEL PERFORMANCE BY POSITION")
	print("-" * 50)
	
	# Extract metrics for each position
	positions = list(position_models.keys())
	train_r2_scores = []
	test_r2_scores = []
	train_mae_scores = []
	test_mae_scores = []
	
	for pos in positions:
		metrics = position_models[pos]['metrics']
		train_r2_scores.append(metrics['train_r2'])
		test_r2_scores.append(metrics['test_r2'])
		train_mae_scores.append(metrics['train_mae'])
		test_mae_scores.append(metrics['test_mae'])
	
	# Create figure with subplots
	fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
	
	# Plot 1: R² Scores by Position
	x = np.arange(len(positions))
	width = 0.35
	
	bars1 = ax1.bar(x - width/2, train_r2_scores, width, label='Train R²', alpha=0.8)
	bars2 = ax1.bar(x + width/2, test_r2_scores, width, label='Test R²', alpha=0.8)
	
	ax1.set_xlabel('Position', fontweight='bold')
	ax1.set_ylabel('R² Score', fontweight='bold')
	ax1.set_title('Random Forest Model Performance - R² by Position', fontweight='bold', fontsize=14)
	ax1.set_xticks(x)
	ax1.set_xticklabels(positions)
	ax1.legend()
	ax1.grid(True, alpha=0.3, axis='y')
	ax1.set_ylim(0, 1.1)
	
	# Add value labels on bars
	for bars in [bars1, bars2]:
		for bar in bars:
			height = bar.get_height()
			ax1.annotate(f'{height:.3f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	# Plot 2: MAE Scores by Position
	bars3 = ax2.bar(x - width/2, train_mae_scores, width, label='Train MAE', alpha=0.8, color='orange')
	bars4 = ax2.bar(x + width/2, test_mae_scores, width, label='Test MAE', alpha=0.8, color='red')
	
	ax2.set_xlabel('Position', fontweight='bold')
	ax2.set_ylabel('Mean Absolute Error', fontweight='bold')
	ax2.set_title('Random Forest Model Performance - MAE by Position', fontweight='bold', fontsize=14)
	ax2.set_xticks(x)
	ax2.set_xticklabels(positions)
	ax2.legend()
	ax2.grid(True, alpha=0.3, axis='y')
	
	# Add value labels on bars
	for bars in [bars3, bars4]:
		for bar in bars:
			height = bar.get_height()
			ax2.annotate(f'{height:.2f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	plt.suptitle('Random Forest Model Performance Analysis by Position', fontsize=16, fontweight='bold', y=1.02)
	plt.tight_layout()
	plt.show()
	
	# Print summary statistics
	print("\n📊 RANDOM FOREST MODEL SUMMARY:")
	print("-" * 60)
	print(f"{'Position':<15} {'Train R²':<10} {'Test R²':<10} {'Train MAE':<10} {'Test MAE':<10}")
	print("-" * 60)
	for i, pos in enumerate(positions):
		print(f"{pos:<15} {train_r2_scores[i]:<10.3f} {test_r2_scores[i]:<10.3f} {train_mae_scores[i]:<10.2f} {test_mae_scores[i]:<10.2f}")
	
	# Calculate average performance
	avg_test_r2 = np.mean(test_r2_scores)
	avg_test_mae = np.mean(test_mae_scores)
	print("-" * 60)
	print(f"{'Average':<15} {np.mean(train_r2_scores):<10.3f} {avg_test_r2:<10.3f} {np.mean(train_mae_scores):<10.2f} {avg_test_mae:<10.2f}")

# Check if we have XGBoost models
if 'xgboost_models' in locals() and xgboost_models:
	print("\n\n📊 PLOTTING XGBOOST MODEL PERFORMANCE BY POSITION")
	print("-" * 50)
	
	# Extract metrics for each position
	xgb_positions = list(xgboost_models.keys())
	xgb_test_r2_scores = []
	xgb_test_mae_scores = []
	xgb_val_r2_scores = []
	xgb_val_mae_scores = []
	
	for pos in xgb_positions:
		metrics = xgboost_models[pos]['model_info']['metrics']
		xgb_test_r2_scores.append(metrics['test_r2'])
		xgb_test_mae_scores.append(metrics['test_mae'])
		xgb_val_r2_scores.append(metrics['val_r2'])
		xgb_val_mae_scores.append(metrics['val_mae'])
	
	# Create figure with subplots
	fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
	
	# Plot 1: R² Scores by Position
	x = np.arange(len(xgb_positions))
	width = 0.35
	
	bars1 = ax1.bar(x - width/2, xgb_val_r2_scores, width, label='Validation R²', alpha=0.8, color='lightgreen')
	bars2 = ax1.bar(x + width/2, xgb_test_r2_scores, width, label='Test R²', alpha=0.8, color='darkgreen')
	
	ax1.set_xlabel('Model', fontweight='bold')
	ax1.set_ylabel('R² Score', fontweight='bold')
	ax1.set_title('XGBoost Model Performance - R² by Position', fontweight='bold', fontsize=14)
	ax1.set_xticks(x)
	ax1.set_xticklabels(xgb_positions)
	ax1.legend()
	ax1.grid(True, alpha=0.3, axis='y')
	ax1.set_ylim(0.8, 1.02)
	
	# Add value labels on bars
	for bars in [bars1, bars2]:
		for bar in bars:
			height = bar.get_height()
			ax1.annotate(f'{height:.3f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	# Plot 2: MAE Scores by Position
	bars3 = ax2.bar(x - width/2, xgb_val_mae_scores, width, label='Validation MAE', alpha=0.8, color='lightcoral')
	bars4 = ax2.bar(x + width/2, xgb_test_mae_scores, width, label='Test MAE', alpha=0.8, color='darkred')
	
	ax2.set_xlabel('Model', fontweight='bold')
	ax2.set_ylabel('Mean Absolute Error', fontweight='bold')
	ax2.set_title('XGBoost Model Performance - MAE by Position', fontweight='bold', fontsize=14)
	ax2.set_xticks(x)
	ax2.set_xticklabels(xgb_positions)
	ax2.legend()
	ax2.grid(True, alpha=0.3, axis='y')
	
	# Add value labels on bars
	for bars in [bars3, bars4]:
		for bar in bars:
			height = bar.get_height()
			ax2.annotate(f'{height:.2f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	plt.suptitle('XGBoost Model Performance Analysis', fontsize=16, fontweight='bold', y=1.02)
	plt.tight_layout()
	plt.show()
	
	# Print summary statistics
	print("\n📊 XGBOOST MODEL SUMMARY:")
	print("-" * 70)
	print(f"{'Model':<15} {'Val R²':<10} {'Test R²':<10} {'Val MAE':<10} {'Test MAE':<10}")
	print("-" * 70)
	for i, pos in enumerate(xgb_positions):
		print(f"{pos:<15} {xgb_val_r2_scores[i]:<10.3f} {xgb_test_r2_scores[i]:<10.3f} {xgb_val_mae_scores[i]:<10.2f} {xgb_test_mae_scores[i]:<10.2f}")
	
	# Calculate average performance
	avg_test_r2 = np.mean(xgb_test_r2_scores)
	avg_test_mae = np.mean(xgb_test_mae_scores)
	print("-" * 70)
	print(f"{'Average':<15} {np.mean(xgb_val_r2_scores):<10.3f} {avg_test_r2:<10.3f} {np.mean(xgb_val_mae_scores):<10.2f} {avg_test_mae:<10.2f}")

# Comparison plot between RandomForest and XGBoost (if both exist)
if 'position_models' in locals() and 'xgboost_models' in locals() and position_models and xgboost_models:
	print("\n\n📊 COMPARING RANDOM FOREST VS XGBOOST PERFORMANCE")
	print("-" * 50)
	
	# Find common positions
	rf_positions = set(position_models.keys())
	xgb_positions = set(xgboost_models.keys()) - {'Combined'}  # Exclude combined model
	common_positions = list(rf_positions.intersection(xgb_positions))
	
	if common_positions:
		# Extract test R² scores for comparison
		rf_test_r2 = [position_models[pos]['metrics']['test_r2'] for pos in common_positions]
		xgb_test_r2 = [xgboost_models[pos]['model_info']['metrics']['test_r2'] for pos in common_positions]
		
		rf_test_mae = [position_models[pos]['metrics']['test_mae'] for pos in common_positions]
		xgb_test_mae = [xgboost_models[pos]['model_info']['metrics']['test_mae'] for pos in common_positions]
		
		# Create comparison plot
		fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
		
		# R² Comparison
		x = np.arange(len(common_positions))
		width = 0.35
		
		bars1 = ax1.bar(x - width/2, rf_test_r2, width, label='Random Forest', alpha=0.8, color='steelblue')
		bars2 = ax1.bar(x + width/2, xgb_test_r2, width, label='XGBoost', alpha=0.8, color='darkgreen')
		
		ax1.set_xlabel('Position', fontweight='bold')
		ax1.set_ylabel('Test R² Score', fontweight='bold')
		ax1.set_title('Model Comparison - Test R² by Position', fontweight='bold', fontsize=14)
		ax1.set_xticks(x)
		ax1.set_xticklabels(common_positions)
		ax1.legend()
		ax1.grid(True, alpha=0.3, axis='y')
		ax1.set_ylim(0.8, 1.02)
		
		# Add value labels
		for bars in [bars1, bars2]:
			for bar in bars:
				height = bar.get_height()
				ax1.annotate(f'{height:.3f}',
							xy=(bar.get_x() + bar.get_width() / 2, height),
							xytext=(0, 3),
							textcoords="offset points",
							ha='center', va='bottom',
							fontsize=9)
		
		# MAE Comparison
		bars3 = ax2.bar(x - width/2, rf_test_mae, width, label='Random Forest', alpha=0.8, color='steelblue')
		bars4 = ax2.bar(x + width/2, xgb_test_mae, width, label='XGBoost', alpha=0.8, color='darkgreen')
		
		ax2.set_xlabel('Position', fontweight='bold')
		ax2.set_ylabel('Test MAE', fontweight='bold')
		ax2.set_title('Model Comparison - Test MAE by Position', fontweight='bold', fontsize=14)
		ax2.set_xticks(x)
		ax2.set_xticklabels(common_positions)
		ax2.legend()
		ax2.grid(True, alpha=0.3, axis='y')
		
		# Add value labels
		for bars in [bars3, bars4]:
			for bar in bars:
				height = bar.get_height()
				ax2.annotate(f'{height:.2f}',
							xy=(bar.get_x() + bar.get_width() / 2, height),
							xytext=(0, 3),
							textcoords="offset points",
							ha='center', va='bottom',
							fontsize=9)
		
		plt.suptitle('Random Forest vs XGBoost Model Comparison', fontsize=16, fontweight='bold', y=1.02)
		plt.tight_layout()
		plt.show()
		
		# Print comparison summary
		print("\n📊 MODEL COMPARISON SUMMARY:")
		print("-" * 80)
		print(f"{'Position':<15} {'RF Test R²':<12} {'XGB Test R²':<12} {'RF Test MAE':<12} {'XGB Test MAE':<12}")
		print("-" * 80)
		for i, pos in enumerate(common_positions):
			print(f"{pos:<15} {rf_test_r2[i]:<12.3f} {xgb_test_r2[i]:<12.3f} {rf_test_mae[i]:<12.2f} {xgb_test_mae[i]:<12.2f}")
		
		# Calculate averages and improvements
		avg_rf_r2 = np.mean(rf_test_r2)
		avg_xgb_r2 = np.mean(xgb_test_r2)
		avg_rf_mae = np.mean(rf_test_mae)
		avg_xgb_mae = np.mean(xgb_test_mae)
		
		print("-" * 80)
		print(f"{'Average':<15} {avg_rf_r2:<12.3f} {avg_xgb_r2:<12.3f} {avg_rf_mae:<12.2f} {avg_xgb_mae:<12.2f}")
		print("-" * 80)
		
		# Calculate improvements
		r2_improvement = ((avg_xgb_r2 - avg_rf_r2) / avg_rf_r2) * 100
		mae_improvement = ((avg_rf_mae - avg_xgb_mae) / avg_rf_mae) * 100
		
		print(f"\n🎯 XGBoost Performance vs Random Forest:")
		print(f"   R² Score: {'↑' if r2_improvement > 0 else '↓'} {abs(r2_improvement):.1f}% {'better' if r2_improvement > 0 else 'worse'}")
		print(f"   MAE:      {'↓' if mae_improvement > 0 else '↑'} {abs(mae_improvement):.1f}% {'better' if mae_improvement > 0 else 'worse'}")
		
		# Determine overall winner
		if r2_improvement > 0 and mae_improvement > 0:
			print(f"\n🏆 Winner: XGBoost (better on both metrics)")
		elif r2_improvement < 0 and mae_improvement < 0:
			print(f"\n🏆 Winner: Random Forest (better on both metrics)")
		else:
			print(f"\n🏆 Winner: Mixed results - depends on metric priority")

else:
	print("⚠️ No model data available for plotting. Please ensure position_models and/or xgboost_models are defined.")

import matplotlib.pyplot as plt
import numpy as np

# Check if we have the position models from RandomForest
if 'position_models' in locals() and position_models:
	print("📊 PLOTTING RANDOM FOREST MODEL PERFORMANCE BY POSITION")
	print("-" * 50)
	
	# Extract metrics for each position
	positions = list(position_models.keys())
	train_r2_scores = []
	test_r2_scores = []
	train_mae_scores = []
	test_mae_scores = []
	
	for pos in positions:
		metrics = position_models[pos]['metrics']
		train_r2_scores.append(metrics['train_r2'])
		test_r2_scores.append(metrics['test_r2'])
		train_mae_scores.append(metrics['train_mae'])
		test_mae_scores.append(metrics['test_mae'])
	
	# Create figure with subplots
	fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
	
	# Plot 1: R² Scores by Position
	x = np.arange(len(positions))
	width = 0.35
	
	bars1 = ax1.bar(x - width/2, train_r2_scores, width, label='Train R²', alpha=0.8)
	bars2 = ax1.bar(x + width/2, test_r2_scores, width, label='Test R²', alpha=0.8)
	
	ax1.set_xlabel('Position', fontweight='bold')
	ax1.set_ylabel('R² Score', fontweight='bold')
	ax1.set_title('Random Forest Model Performance - R² by Position', fontweight='bold', fontsize=14)
	ax1.set_xticks(x)
	ax1.set_xticklabels(positions)
	ax1.legend()
	ax1.grid(True, alpha=0.3, axis='y')
	ax1.set_ylim(0, 1.1)
	
	# Add value labels on bars
	for bars in [bars1, bars2]:
		for bar in bars:
			height = bar.get_height()
			ax1.annotate(f'{height:.3f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	# Plot 2: MAE Scores by Position
	bars3 = ax2.bar(x - width/2, train_mae_scores, width, label='Train MAE', alpha=0.8, color='orange')
	bars4 = ax2.bar(x + width/2, test_mae_scores, width, label='Test MAE', alpha=0.8, color='red')
	
	ax2.set_xlabel('Position', fontweight='bold')
	ax2.set_ylabel('Mean Absolute Error', fontweight='bold')
	ax2.set_title('Random Forest Model Performance - MAE by Position', fontweight='bold', fontsize=14)
	ax2.set_xticks(x)
	ax2.set_xticklabels(positions)
	ax2.legend()
	ax2.grid(True, alpha=0.3, axis='y')
	
	# Add value labels on bars
	for bars in [bars3, bars4]:
		for bar in bars:
			height = bar.get_height()
			ax2.annotate(f'{height:.2f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	plt.suptitle('Random Forest Model Performance Analysis by Position', fontsize=16, fontweight='bold', y=1.02)
	plt.tight_layout()
	plt.show()
	
	# Print summary statistics
	print("\n📊 RANDOM FOREST MODEL SUMMARY:")
	print("-" * 60)
	print(f"{'Position':<15} {'Train R²':<10} {'Test R²':<10} {'Train MAE':<10} {'Test MAE':<10}")
	print("-" * 60)
	for i, pos in enumerate(positions):
		print(f"{pos:<15} {train_r2_scores[i]:<10.3f} {test_r2_scores[i]:<10.3f} {train_mae_scores[i]:<10.2f} {test_mae_scores[i]:<10.2f}")
	
	# Calculate average performance
	avg_test_r2 = np.mean(test_r2_scores)
	avg_test_mae = np.mean(test_mae_scores)
	print("-" * 60)
	print(f"{'Average':<15} {np.mean(train_r2_scores):<10.3f} {avg_test_r2:<10.3f} {np.mean(train_mae_scores):<10.2f} {avg_test_mae:<10.2f}")

# Check if we have XGBoost models
if 'xgboost_models' in locals() and xgboost_models:
	print("\n\n📊 PLOTTING XGBOOST MODEL PERFORMANCE BY POSITION")
	print("-" * 50)
	
	# Extract metrics for each position
	xgb_positions = list(xgboost_models.keys())
	xgb_test_r2_scores = []
	xgb_test_mae_scores = []
	xgb_val_r2_scores = []
	xgb_val_mae_scores = []
	
	for pos in xgb_positions:
		metrics = xgboost_models[pos]['model_info']['metrics']
		xgb_test_r2_scores.append(metrics['test_r2'])
		xgb_test_mae_scores.append(metrics['test_mae'])
		xgb_val_r2_scores.append(metrics['val_r2'])
		xgb_val_mae_scores.append(metrics['val_mae'])
	
	# Create figure with subplots
	fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
	
	# Plot 1: R² Scores by Position
	x = np.arange(len(xgb_positions))
	width = 0.35
	
	bars1 = ax1.bar(x - width/2, xgb_val_r2_scores, width, label='Validation R²', alpha=0.8, color='lightgreen')
	bars2 = ax1.bar(x + width/2, xgb_test_r2_scores, width, label='Test R²', alpha=0.8, color='darkgreen')
	
	ax1.set_xlabel('Model', fontweight='bold')
	ax1.set_ylabel('R² Score', fontweight='bold')
	ax1.set_title('XGBoost Model Performance - R² by Position', fontweight='bold', fontsize=14)
	ax1.set_xticks(x)
	ax1.set_xticklabels(xgb_positions)
	ax1.legend()
	ax1.grid(True, alpha=0.3, axis='y')
	ax1.set_ylim(0.8, 1.02)
	
	# Add value labels on bars
	for bars in [bars1, bars2]:
		for bar in bars:
			height = bar.get_height()
			ax1.annotate(f'{height:.3f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	# Plot 2: MAE Scores by Position
	bars3 = ax2.bar(x - width/2, xgb_val_mae_scores, width, label='Validation MAE', alpha=0.8, color='lightcoral')
	bars4 = ax2.bar(x + width/2, xgb_test_mae_scores, width, label='Test MAE', alpha=0.8, color='darkred')
	
	ax2.set_xlabel('Model', fontweight='bold')
	ax2.set_ylabel('Mean Absolute Error', fontweight='bold')
	ax2.set_title('XGBoost Model Performance - MAE by Position', fontweight='bold', fontsize=14)
	ax2.set_xticks(x)
	ax2.set_xticklabels(xgb_positions)
	ax2.legend()
	ax2.grid(True, alpha=0.3, axis='y')
	
	# Add value labels on bars
	for bars in [bars3, bars4]:
		for bar in bars:
			height = bar.get_height()
			ax2.annotate(f'{height:.2f}',
						xy=(bar.get_x() + bar.get_width() / 2, height),
						xytext=(0, 3),
						textcoords="offset points",
						ha='center', va='bottom',
						fontsize=9)
	
	plt.suptitle('XGBoost Model Performance Analysis', fontsize=16, fontweight='bold', y=1.02)
	plt.tight_layout()
	plt.show()
	
	# Print summary statistics
	print("\n📊 XGBOOST MODEL SUMMARY:")
	print("-" * 70)
	print(f"{'Model':<15} {'Val R²':<10} {'Test R²':<10} {'Val MAE':<10} {'Test MAE':<10}")
	print("-" * 70)
	for i, pos in enumerate(xgb_positions):
		print(f"{pos:<15} {xgb_val_r2_scores[i]:<10.3f} {xgb_test_r2_scores[i]:<10.3f} {xgb_val_mae_scores[i]:<10.2f} {xgb_test_mae_scores[i]:<10.2f}")
	
	# Calculate average performance
	avg_test_r2 = np.mean(xgb_test_r2_scores)
	avg_test_mae = np.mean(xgb_test_mae_scores)
	print("-" * 70)
	print(f"{'Average':<15} {np.mean(xgb_val_r2_scores):<10.3f} {avg_test_r2:<10.3f} {np.mean(xgb_val_mae_scores):<10.2f} {avg_test_mae:<10.2f}")

# Comparison plot between RandomForest and XGBoost (if both exist)
if 'position_models' in locals() and 'xgboost_models' in locals() and position_models and xgboost_models:
	print("\n\n📊 COMPARING RANDOM FOREST VS XGBOOST PERFORMANCE")
	print("-" * 50)
	
	# Find common positions
	rf_positions = set(position_models.keys())
	xgb_positions = set(xgboost_models.keys()) - {'Combined'}  # Exclude combined model
	common_positions = list(rf_positions.intersection(xgb_positions))
	
	if common_positions:
		# Extract test R² scores for comparison
		rf_test_r2 = [position_models[pos]['metrics']['test_r2'] for pos in common_positions]
		xgb_test_r2 = [xgboost_models[pos]['model_info']['metrics']['test_r2'] for pos in common_positions]
		
		rf_test_mae = [position_models[pos]['metrics']['test_mae'] for pos in common_positions]
		xgb_test_mae = [xgboost_models[pos]['model_info']['metrics']['test_mae'] for pos in common_positions]
		
		# Create comparison plot
		fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
		
		# R² Comparison
		x = np.arange(len(common_positions))
		width = 0.35
		
		bars1 = ax1.bar(x - width/2, rf_test_r2, width, label='Random Forest', alpha=0.8, color='steelblue')
		bars2 = ax1.bar(x + width/2, xgb_test_r2, width, label='XGBoost', alpha=0.8, color='darkgreen')
		
		ax1.set_xlabel('Position', fontweight='bold')
		ax1.set_ylabel('Test R² Score', fontweight='bold')
		ax1.set_title('Model Comparison - Test R² by Position', fontweight='bold', fontsize=14)
		ax1.set_xticks(x)
		ax1.set_xticklabels(common_positions)
		ax1.legend()
		ax1.grid(True, alpha=0.3, axis='y')
		ax1.set_ylim(0.8, 1.02)
		
		# Add value labels
		for bars in [bars1, bars2]:
			for bar in bars:
				height = bar.get_height()
				ax1.annotate(f'{height:.3f}',
							xy=(bar.get_x() + bar.get_width() / 2, height),
							xytext=(0, 3),
							textcoords="offset points",
							ha='center', va='bottom',
							fontsize=9)
		
		# MAE Comparison
		bars3 = ax2.bar(x - width/2, rf_test_mae, width, label='Random Forest', alpha=0.8, color='steelblue')
		bars4 = ax2.bar(x + width/2, xgb_test_mae, width, label='XGBoost', alpha=0.8, color='darkgreen')
		
		ax2.set_xlabel('Position', fontweight='bold')
		ax2.set_ylabel('Test MAE', fontweight='bold')
		ax2.set_title('Model Comparison - Test MAE by Position', fontweight='bold', fontsize=14)
		ax2.set_xticks(x)
		ax2.set_xticklabels(common_positions)
		ax2.legend()
		ax2.grid(True, alpha=0.3, axis='y')
		
		# Add value labels
		for bars in [bars3, bars4]:
			for bar in bars:
				height = bar.get_height()
				ax2.annotate(f'{height:.2f}',
							xy=(bar.get_x() + bar.get_width() / 2, height),
							xytext=(0, 3),
							textcoords="offset points",
							ha='center', va='bottom',
							fontsize=9)
		
		plt.suptitle('Random Forest vs XGBoost Model Comparison', fontsize=16, fontweight='bold', y=1.02)
		plt.tight_layout()
		plt.show()
		
		# Print comparison summary
		print("\n📊 MODEL COMPARISON SUMMARY:")
		print("-" * 80)
		print(f"{'Position':<15} {'RF Test R²':<12} {'XGB Test R²':<12} {'RF Test MAE':<12} {'XGB Test MAE':<12}")
		print("-" * 80)
		for i, pos in enumerate(common_positions):
			print(f"{pos:<15} {rf_test_r2[i]:<12.3f} {xgb_test_r2[i]:<12.3f} {rf_test_mae[i]:<12.2f} {xgb_test_mae[i]:<12.2f}")
		
		# Calculate averages and improvements
		avg_rf_r2 = np.mean(rf_test_r2)
		avg_xgb_r2 = np.mean(xgb_test_r2)
		avg_rf_mae = np.mean(rf_test_mae)
		avg_xgb_mae = np.mean(xgb_test_mae)
		
		print("-" * 80)
		print(f"{'Average':<15} {avg_rf_r2:<12.3f} {avg_xgb_r2:<12.3f} {avg_rf_mae:<12.2f} {avg_xgb_mae:<12.2f}")
		print("-" * 80)
		
		# Calculate improvements
		r2_improvement = ((avg_xgb_r2 - avg_rf_r2) / avg_rf_r2) * 100
		mae_improvement = ((avg_rf_mae - avg_xgb_mae) / avg_rf_mae) * 100
		
		print(f"\n🎯 XGBoost Performance vs Random Forest:")
		print(f"   R² Score: {'↑' if r2_improvement > 0 else '↓'} {abs(r2_improvement):.1f}% {'better' if r2_improvement > 0 else 'worse'}")
		print(f"   MAE:      {'↓' if mae_improvement > 0 else '↑'} {abs(mae_improvement):.1f}% {'better' if mae_improvement > 0 else 'worse'}")
		
		# Determine overall winner
		if r2_improvement > 0 and mae_improvement > 0:
			print(f"\n🏆 Winner: XGBoost (better on both metrics)")
		elif r2_improvement < 0 and mae_improvement < 0:
			print(f"\n🏆 Winner: Random Forest (better on both metrics)")
		else:
			print(f"\n🏆 Winner: Mixed results - depends on metric priority")

else:
	print("⚠️ No model data available for plotting. Please ensure position_models and/or xgboost_models are defined.")
